# 0 Functions

In [14]:
import sys, traceback
print(sys.executable)
print(sys.version)
print(sys.version_info)
from multiprocessing import Pool, cpu_count
from concurrent.futures import ProcessPoolExecutor

import math
import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from spectral_cube import LazyMask
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              

import astrodendro #Change numpy.int to int, modified asscalar to just take an element
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
import gc
from astropy.convolution import Gaussian1DKernel
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astrodendro.analysis import PPVStatistic
import os
print(spectral_cube.__file__)

print(astrodendro.__file__)

from matplotlib.patches import Ellipse
import copy
from tqdm import tqdm
import radio_beam
from astropy.table import Table
print(np.__version__,"Numpy")
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
#%matplotlib widget
Num_per_kg= 6.0221409*10**23/(2.8*10**-3)#6.0221409*10**23/29.0180*10**-3#num/kg for h2

#Create a function that uses the dendrogram input to calculate all the quantities, and has the size and linewidth requirements of the Shetty paper
#Requires the computed dendrogram, the data from the line image, the velocity axis, and the data from the Continuum image, as well as metadata for the structures
#Finds Size, Linewidth, Luminosity, and Column Density of a structure for each structure and returns them in [[][]] arrays [Leaves][Branches]
#Continuum is in Jansky/Beam, Line data should have the unit specified in the metadata as 'data_unit'
gal="GC"

dist_cmz = 8.178*10**-3*u.Mpc

def Dendro_Arrays(Dendrogram,LineData,DataVel,ContData,metadata,ColD = True,beam_size=999,beam_req = 999999,Trunks=True,max_size=18,edge_cases=False):
    SizeA,SigmaA,LuminA,CDA,SIDS,MOM0_FLUX,Distances,V_rms_err = [[],[],[],[]],[[],[],[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
    print(metadata)
    
    d_copy= Dendrogram
    #catalog = astrodendro.ppv_catalog(d, metadata)
    if (gal =="GC"):
        center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
        dist_val=8.178*10**3
    else:
        center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')
        dist_val=3.5*10**6
    
    center_ra_pix,center_dec_pix = int(metadata['wcsu'][:][:][0].world_to_pixel(center)[0]),int(metadata['wcsu'][:][:][0].world_to_pixel(center)[1])
    
    sliced= LineData[12]
    CubeShape = np.shape(sliced)
    DataShape=[[0,0],[0,0]]#The part of the cube that actually has data

    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[lmi])
        if(allData>0 or allData<0):
            DataShape[0][0] = lmi+3
            break
    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[CubeShape[0] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[0][1] = CubeShape[0] - lmi -3
            break
    for lmi in range(CubeShape[1]):
        allData=(sliced[DataShape[0][0],lmi])
        if(allData>0 or allData<0):
            DataShape[1][0] = lmi+3
            break
    for lmi in range(CubeShape[1]):
        allData=(sliced[DataShape[0][0],CubeShape[1] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[1][1] = CubeShape[1] - lmi -3
            break
    for t in Dendrogram.all_structures: 

        I = t.indices()
        Cont = True
        if t.is_branch:
                if t.parent==None:
                    
                    if(Trunks):
                        Cont = True
                    else:
                        Cont = False
                else:
                    Cont=True
                    
        for lmi in range(len(I[0])):
            if(I[1][lmi]<=DataShape[0][0] or I[1][lmi]>=DataShape[0][1] or I[2][lmi]<=DataShape[1][0] or I[2][lmi]>=DataShape[1][1]):
                #print(I[1][lmi],I[0][lmi])
                if edge_cases:
                    Cont=True
                else:
                    Cont=False
                break
                

        if(Cont):
            s = PPVStatistic(t,metadata=metadata)
            s_radius = s.radius
            s_v_rms = s.v_rms
            
            
            if((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*dist_val/10**6/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01):
            
            

                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                
                #The flux from the continuum
                #Convert to Jansky from Jansky per beam:
                if(ColD ==True):
                    Cont_Flux=0

                    proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                    for lmi in range(len(proj)):

                        Cont_Flux+=ContData[proj[lmi]]
                    Cont_Flux=Cont_Flux/(metadata['beam_area_ratioc']*(2*np.sqrt(2*np.log(2))))*u.pix**2*u.beam/u.beam*u.Jy#SHould be input as Jansky /beam and will be converted to Jansky, then to unitless. The beam is changed from FWHM to Gaussian
                    Dust_Column = Flux_to_Mass(Cont_Flux)*Num_per_kg/((s_radius*np.pi/180*dist_cmz.value/u.deg)**2*(3.086*10**24)**2)/np.pi*(1.989*10**30*u.kg/u.M_sun)/u.kg
                    
                else:
                    Dust_Column=0
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"])#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                
                
                
                Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2)*metadata['spatial_scale']*np.pi/180*dist_cmz.value*10**6/u.deg#pc dist from barycenter
                
                
                V_err= Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m,NF=1,iterations=5,metadata=metadata)
                
                
                if(t.is_leaf):

                    SizeA[0].append((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))) #define size as astrodendro
                    SigmaA[0].append((float(s_v_rms/u.km*u.s)))#
                    CDA[0].append(float(Dust_Column))
                    LuminA[0].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[0].append(float(t.idx))
                    MOM0_FLUX[0].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[0].append(float(Distance))
                    V_rms_err[0].append(float(V_err))
                if(t.is_branch	):

                    SizeA[1].append((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))) #define size as astrodendro
                    SigmaA[1].append((float(s_v_rms/u.km*u.s)))#
                    CDA[1].append(float(Dust_Column))
                    LuminA[1].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[1].append(float(t.idx))
                    MOM0_FLUX[1].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[1].append(float(Distance))
                    V_rms_err[1].append(float(V_err))
                del s
                    
                    
                    
    SizeA[0] = np.array(SizeA[0],dtype=type(1.))
    SizeA[1] = np.array(SizeA[1],dtype=type(1.))
    SizeA[2] = np.array(SizeA[2],dtype=type(1.))
    SizeA[3] = np.array(SizeA[3],dtype=type(1.))
    SigmaA[0] = np.array(SigmaA[0],dtype=type(1.))
    SigmaA[1] = np.array(SigmaA[1],dtype=type(1.))
    SigmaA[2] = np.array(SigmaA[2],dtype=type(1.))
    SigmaA[3] = np.array(SigmaA[3],dtype=type(1.))
    CDA[0] = np.array(CDA[0],dtype=type(1.))
    CDA[1] = np.array(CDA[1],dtype=type(1.))
    LuminA[0] = np.array(LuminA[0],dtype=type(1.))
    LuminA[1] = np.array(LuminA[1],dtype=type(1.))
    SIDS[0] = np.array(SIDS[0],dtype=type(1.))
    SIDS[1] = np.array(SIDS[1],dtype=type(1.))
    MOM0_FLUX[0] = np.array(MOM0_FLUX[0],dtype=type(1.))
    MOM0_FLUX[1] = np.array(MOM0_FLUX[1],dtype=type(1.))
    Distances[0] = np.array(Distances[0],dtype=type(1.))
    Distances[1] = np.array(Distances[1],dtype=type(1.))
    V_rms_err[0] = np.array(V_rms_err[0],dtype=type(1.))
    V_rms_err[1] = np.array(V_rms_err[1],dtype=type(1.))
    
    return np.array(SizeA),np.array(SigmaA),np.array(CDA),np.array(LuminA),np.array(SIDS),np.array(MOM0_FLUX),np.array(Distances),np.array(V_rms_err)

#Make a function to make an image 

#Data to plot, minimum of color bar, maximum, WCS projection for coords, and position of the image in the larger figure
def Make_Plot(Name,Name2,Data,vmin,vmax,WCS,rows,columns,index,show):
    

    ax = pylab.subplot(rows,columns,index,projection=WCS) 
    RA = ax.coords[0]                                                                  # 
    Dec = ax.coords[1]
    im = pylab.imshow(Data,vmin=vmin,vmax=vmax,cmap='rainbow')
    RA.set_ticks(size=-3)                                                                                      
    Dec.set_ticks(size=-3) 
    RA.set_ticklabel(exclude_overlapping=True) 
    Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
    
    if(gal=="NGC253"):
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
    else:
        pylab.xlabel('Glon',fontsize=20,labelpad=1)                               
        pylab.ylabel('Glat',fontsize=20,labelpad=1)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 15)    
    cb=pylab.colorbar(im,fraction=0.1,pad=0.0)                                     
    cb.set_label(label=Name,fontsize=10,rotation=270,labelpad=20) 
    cb.ax.tick_params(which = 'major', labelsize = 10)   
    pylab.annotate(text=Name2,fontsize=10,xy=(0.02,1.05),xycoords="axes fraction")  
    if(show==True):
        pylab.show()
def Make_Plot_Anno(Name,Name2,Data,vmin,vmax,WCS,rows,columns,index,show,pos1,pos2):
    

    ax = pylab.subplot(rows,columns,index,projection=WCS) 
    RA = ax.coords[0]                                                                  # 
    Dec = ax.coords[1]
    im = pylab.imshow(Data,vmin=vmin,vmax=vmax,cmap='rainbow')
    RA.set_ticks(size=-3)                                                                                      
    Dec.set_ticks(size=-3) 
    RA.set_ticklabel(exclude_overlapping=True) 
    Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
    if(gal=="NGC253"):
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
    else:
        pylab.xlabel('Glon',fontsize=20,labelpad=1)                               
        pylab.ylabel('Glat',fontsize=20,labelpad=1)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 15)    
    cb=pylab.colorbar(im,fraction=.1,pad=0.0)                                     
    cb.set_label(label=Name,fontsize=10,rotation=270,labelpad=20) 
    cb.ax.tick_params(which = 'major', labelsize = 10)   
    pylab.annotate(text=Name2,fontsize=10,xy=(pos1,pos2),xycoords="axes fraction")  
    if(show==True):
        pylab.show()
        
        
#Put this up here for the column density map
def Flux_to_Mass(flux):
    J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy
    flux_erg = flux*J_to_e
    if(gal=="NGC253"):
        L = 4*np.pi*(3.5*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    else:
        L = 4*np.pi*(8.178*10**-3*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    
    
    a_850 = 6.7*10**19*u.erg/u.s/u.Hz/u.M_sun #6.7+-1.7
    
    M_mol = L/a_850#Just in Solar mass*1.989*10**30*u.kg/u.M_sun #Determines mass of the cont for 850 in kg
    return M_mol
def Flux_to_Lum(flux):
    J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy
    flux_erg = flux*J_to_e
    if(gal=="NGC253"):
        L = 4*np.pi*(3.5*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    else:
        L = 4*np.pi*(8.178*10**-3*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    
    return L


def Get_V_rms_err(dend1,struct,idx,m,NF,iterations,metadata):
    
    
    vs=[]
    np.random.seed((99)**2*123)
    for llll in range(iterations):
        
        #print(llll)
        s = dend1.__getitem__(idx)
        #s = struct#copy.deepcopy(struct)
        #s2 = struct#copy.deepcopy(struct)
        npixels = np.product(np.shape(s.values()))
        #print(np.shape(s.values()),s.values())
        
        additional_noise = np.random.normal(0., m*NF, npixels)
        additional_noise = np.reshape(additional_noise, np.shape(s.values()))
        #add or subract noise to the values and calculate the v rms, them find the std of that array and
        # call that the uncertainty in v rms for a structure
        dat1P = dend1.data[s.indices()]
        dend1.data[s.indices()]+= additional_noise
        s = dend1.__getitem__(idx)
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms/u.km*u.s))
        dend1.data[s.indices()]= dat1P#reset the dend data
        
        dend1.data[s.indices()]-= additional_noise
        #s._values+=additional_noise
        #print(s.values(),s._values)
        
        #s2._values-=additional_noise
        s = dend1.__getitem__(idx)
        #print(dat1P[0],s._values[0],"kaasl")
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms/u.km*u.s))
        
        del s
        #del s2
        
    v_rms_std = np.nanstd(vs)
    #print(v_rms_std)
    return v_rms_std

#Return a cropped cube for some ra and dec, also crops the velocity axis if needed (0 for no crop)
def Crop(cube,WCS,Np1,Np2,BadVel,D2):
    NraDP1 = [int(WCS.world_to_pixel(Np1)[0]),int(WCS.world_to_pixel(Np1)[1])]
    NraDP2 = [int(WCS.world_to_pixel(Np2)[0]),int(WCS.world_to_pixel(Np2)[1])]
    if(D2==False):
        return cube[BadVel:np.shape(cube)[0]-BadVel,NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]
    if(D2==True):
        return cube[NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]

    
def Read_Clusters(FileName):
    
    sh= len(np.genfromtxt(FileName,usecols=0))
    Data=[]
    for lmi in range(50):
        try:
            Data.append(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m")))
            #print(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m"),skip_header=1))
        except:
            pass
    return Data
def Find_Clusters_NGC(Data):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
            
    return IDs,RAs,Decs,R_deconv
#Take the cont in Jy and find the HWHM from the structures in the catalog
def Find_Clusters(Data,wcs,Cont_Data,header):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
        if "herschel_column" in Data[lmi]: 
            CD= (Data[lmi][1:9999])#pc
            
        if "flux_integrated" in Data[lmi]: 
            Flux_1p3mm= Data[lmi][1:9999]#pc
    #remove nan 
    for lmii in range(len(CD)):
        try:
            if CD[lmii]=='np.nan':
                CD= np.delete(CD, lmii)
                Flux_1p3mm= np.delete(Flux_1p3mm, lmii)
                IDs= np.delete(IDs, lmii)
                glats= np.delete(glats, lmii)
                glons= np.delete(glons, lmii)
                
        except:
            CD = np.array(CD,dtype=type(1.2**5))#float
            break
    glats_New=[]
    glons_New=[]
    CDs_New=[]
    IDs_New=[]
    Flux_1p3mm_New=[]

    #print(CD,sorted(CD),type(CD),type(CD[0]))
    nth = sorted(CD)[len(CD)-34]#34 most dense leaves
    #print(nth,"A",CD,sorted(CD))
    for lmj in range(len(CD)):
        if CD[lmj]>nth:
            glats_New.append(glats[lmj])
            glons_New.append(glons[lmj])
            CDs_New.append(CD[lmj])
            IDs_New.append(int(IDs[lmj]))
            Flux_1p3mm_New.append(Flux_1p3mm[lmj])
    HWHM_rad = []      
    #print(Flux_1p3mm_New,glats_New,glons_New,CDs_New,IDs_New)
    for lmi in range(len(CDs_New)):
        glat = glats_New[lmi]
        glon = glons_New[lmi]
        Flux = float(Flux_1p3mm_New[lmi])#INtegerated flux in jy
        
        Circle_R = 0
        distance = 8.178*10**-3*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(l=float(glon)*u.deg, b=float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        
        while(True):
            Circle_R += .01
            #pixels=[(p1,p2)]
            pixels=[(p2,p1)]#Goes lat then long for the cont data
            #print(p1,p2)
            #print(np.shape(Cont_Data[p2-50:p2+50]))
            #print(np.shape(Cont_Data[50,p1-50:p1+50]))
            for lmii in range(np.shape(Cont_Data[p2-50:p2+50])[0]):
                for lmjj in range(np.shape(Cont_Data[p2-50+lmii,p1-50:p1+50])[0]):
                    #Find pixels within the circle around the center (excude the center since its there already)
                    #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                    if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                        pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
                        
            
            
            sum_flux=0
            for lmkk in range(len(pixels)):
                sum_flux += (Cont_Data[pixels[lmkk]])
            #print(p1,p2,glat,glon,np.shape(Cont_Data),pixels,Cont_Data[pixels[0]],Flux,sum_flux,Circle_R)
            if sum_flux>Flux/2:
                HWHM_rad.append(Circle_R)#Pc
                break
                
    return HWHM_rad,CDs_New,glons_New,glats_New,IDs_New

#Return masked data around clusters or one pc around clusters
def Mask_Clusters_NGC(HWHM,wcs,header,unmasked_data,ras,decs,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        ra = ras[lmi]
        dec = decs[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = 3.5*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(str(ra),str(dec), frame='icrs')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,1,True)
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,2,True)


def Mask_Clusters_CMZ(HWHM,wcs,header,unmasked_data,glons,glats,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        glon = glons[lmi]
        glat = glats[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = dist_cmz
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(float(glon)*u.deg,float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,1,True)
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,2,True)

def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


from astropy.utils import NumpyRNGContext
def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field
restfreq = 345.79598990 * u.GHz







#For the ngc253 data we need another function

def Dendro_Arrays_NGC(Dendrogram,LineData,DataVel,ContData,metadata,ColD = True,beam_size=999,beam_req = 999999,Edge_Cases=True,Trunks=True,max_size=1):
    SizeA,SigmaA,LuminA,CDA,SIDS,MOM0_FLUX,Distances,V_rms_err = [[],[],[],[]],[[],[],[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
    print(metadata)
    
    d_copy= Dendrogram
    #catalog = astrodendro.ppv_catalog(d, metadata)
    center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')
    center_ra_pix,center_dec_pix = int(metadata['wcsu'][:][:][0].world_to_pixel(center)[0]),int(metadata['wcsu'][:][:][0].world_to_pixel(center)[1])
    sliced= LineData[12]
    CubeShape = np.shape(sliced)
    for t in Dendrogram.all_structures: 

        I = t.indices()
        Cont = True
        if t.is_branch:
                if t.parent==None:
                    if Trunks:
                        Cont=True
                    else:
                        Cont=False
                else:
                    Cont = True
        
        for lmi in range(len(I[0])):
            NansNE=0
            NansSE=0
            NansNW=0
            NansSW=0
            Length = 10
            #I[1][lmi+10]>
            for lmj in range(Length):
                #Check four 45 degree prongs from each point and see if they have at least 7 nans in 10 pixels. If that happens its too close to the boundary
                try:

                    if(sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]<0 ):
                        pass
                    else:
                        NansSE+=1
                    if(sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]<0 ):
                        pass
                    else:
                        NansSW+=1
                    if(sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]<0 ):
                        pass
                    else:
                        NansNW+=1
                    if(sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]<0 ):
                        pass
                    else:
                        NansNE+=1
                except:
                    #only fails if the I goes close to the boundary of the cube and tries to get a pixel outside the cube
                    if not Edge_Cases:
                        Cont = False
            if(NansNE>Length-3 or NansNW>Length-3 or NansSE>Length-3 or NansSW>Length-3):
                if not Edge_Cases:
                    Cont = False
                
                break

        if(Cont):
            s = PPVStatistic(t,metadata=metadata)
            s_radius = s.radius
            s_v_rms = s.v_rms
            if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01):
            
            

                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                
                #The flux from the continuum
                #Convert to Jansky from Jansky per beam:
                if(ColD ==True):
                    Cont_Flux=0

                    proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                    for lmi in range(len(proj)):

                        Cont_Flux+=ContData[proj[lmi]]
                    Cont_Flux=Cont_Flux/(metadata['beam_area_ratioc']*(2*np.sqrt(2*np.log(2))))*u.pix**2*u.beam/u.beam*u.Jy#SHould be input as Jansky /beam and will be converted to Jansky, then to unitless. The beam is changed from FWHM to Gaussian
                    Dust_Column = Flux_to_Mass(Cont_Flux)*Num_per_kg/((s_radius*np.pi/180*3.5/u.deg)**2*(3.086*10**24)**2)/np.pi*(1.989*10**30*u.kg/u.M_sun)/u.kg
                else:
                    Dust_Column=0
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"])#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                
                
                
                Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2)*metadata['spatial_scale']*np.pi/180*3.5*10**6/u.deg#pc dist from barycenter
                
                
                V_err= Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m,NF=1,iterations=5,metadata=metadata)
                
                
                if(t.is_leaf):

                    SizeA[0].append((float((s_radius*np.pi/180*3.5/u.deg)))) #define size as astrodendro
                    SigmaA[0].append((float(s_v_rms/u.km*u.s)))#
                    CDA[0].append(float(Dust_Column))
                    LuminA[0].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[0].append(float(t.idx))
                    MOM0_FLUX[0].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[0].append(float(Distance))
                    V_rms_err[0].append(float(V_err))
                if(t.is_branch	):

                    SizeA[1].append((float((s_radius*np.pi/180*3.5/u.deg)))) #define size as astrodendro
                    SigmaA[1].append((float(s_v_rms/u.km*u.s)))#
                    CDA[1].append(float(Dust_Column))
                    LuminA[1].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[1].append(float(t.idx))
                    MOM0_FLUX[1].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[1].append(float(Distance))
                    V_rms_err[1].append(float(V_err))
                del s
                    
                    
                    
    SizeA[0] = np.array(SizeA[0],dtype=type(1.))
    SizeA[1] = np.array(SizeA[1],dtype=type(1.))
    SizeA[2] = np.array(SizeA[2],dtype=type(1.))
    SizeA[3] = np.array(SizeA[3],dtype=type(1.))
    SigmaA[0] = np.array(SigmaA[0],dtype=type(1.))
    SigmaA[1] = np.array(SigmaA[1],dtype=type(1.))
    SigmaA[2] = np.array(SigmaA[2],dtype=type(1.))
    SigmaA[3] = np.array(SigmaA[3],dtype=type(1.))
    CDA[0] = np.array(CDA[0],dtype=type(1.))
    CDA[1] = np.array(CDA[1],dtype=type(1.))
    LuminA[0] = np.array(LuminA[0],dtype=type(1.))
    LuminA[1] = np.array(LuminA[1],dtype=type(1.))
    SIDS[0] = np.array(SIDS[0],dtype=type(1.))
    SIDS[1] = np.array(SIDS[1],dtype=type(1.))
    MOM0_FLUX[0] = np.array(MOM0_FLUX[0],dtype=type(1.))
    MOM0_FLUX[1] = np.array(MOM0_FLUX[1],dtype=type(1.))
    Distances[0] = np.array(Distances[0],dtype=type(1.))
    Distances[1] = np.array(Distances[1],dtype=type(1.))
    V_rms_err[0] = np.array(V_rms_err[0],dtype=type(1.))
    V_rms_err[1] = np.array(V_rms_err[1],dtype=type(1.))
    
    return np.array(SizeA),np.array(SigmaA),np.array(CDA),np.array(LuminA),np.array(SIDS),np.array(MOM0_FLUX),np.array(Distances),np.array(V_rms_err)




import corner

import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy

import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
from matplotlib.patches import Rectangle
#%matplotlib widget
import matplotlib.gridspec as gridspec
################
# MCMC FITTING #
################

###################################################################################################

# import required modules
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
import scipy.optimize as op
import emcee
import corner


###################################################################################################

# linear fitting function, i.e. two parameter dimensions
def linear(x,a,c):
    return a*x+c

# equivalent power law function
def powlaw(x,a,b):
    return b*np.power(x,a)

# log likelihood
# according to https://ixkael.github.io/fitting-a-line-to-data-a-quick-tutorial/
def lnlikelihood(x, y, xerr, yerr, a, c):

    # only y errors:
    model = linear(x,a,c)
    return -0.5*(np.sum((y-model)**2/yerr**2 - np.log(yerr**2)))

    # # both x and y errors:
    # xyerr = np.sqrt(xerr**2. + yerr**2.)
    # model = linear(x, a, c)
    # return np.sum(-0.5*((y-model)/xyerr)**2 - 0.5*np.log(2*np.pi)-np.log(xyerr))

# negative log likelihood (required to fit maximum likelihood)
# according to https://ixkael.github.io/fitting-a-line-to-data-a-quick-tutorial/
def neg_lnlikelihood(params, x, y, xerr, yerr):
    a, c = params
    return -lnlikelihood(x, y, xerr, yerr, a, c)

# log prior
# simple flat priors on slope and intercept, large range but exclude rediculous values
def lnprior(a,c):
    if ( 0<a<10 ) and ( -10<c<10 ):
        return 0.0
    return -np.inf

# log probability function
# according to https://ixkael.github.io/fitting-a-line-to-data-a-quick-tutorial/
def lnprobability(params, x, y, xerr, yerr):
    a, c = params
    lp = lnprior(a,c)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlikelihood(x, y, xerr, yerr, a, c)


####################################################################################################
# a class to the actual work and fitting
####################################################################################################

class MCMC_fit:

    def __init__(self, x, y, x_err, y_err):
        self.source = None
        self.line   = None
        self.fit    = False
        self.fit_range = [None,None]
        self.x      = np.array(x)
        self.y      = np.array(y)
        self.x_err  = np.array(x_err)
        self.y_err  = np.array(y_err)
        self.ndim = 2
        self.a    = {'ls': None, 'ml': None, 'MCMC': None, 'perc': None}
        self.c    = {'ls': None, 'ml': None, 'MCMC': None, 'perc': None}
        self.nwalkers = 50
        self.nburnin  = 500
        self.nsteps   = 5000
        self.sampler  = None
        self.samples  = None
        self.xlabel   = 'x'
        self.ylabel   = 'y'
        self.figsize  = (6,6)
        self.plotcolor = None
        self.savepath = None

    def mask_bad_values(self):
        print("masking bad values ...")
        good_vals = np.isfinite(self.x) & np.isfinite(self.y) & np.isfinite(self.x_err) & np.isfinite(self.y_err)
        self.x     = self.x[good_vals]
        self.y     = self.y[good_vals]
        self.x_err = self.x_err[good_vals]
        self.y_err = self.y_err[good_vals]

    def restrict_range(self, fit_range=[None,None]):
        self.fit_range = fit_range
        print("restricting fit to "+str(self.fit_range)+" ...")
        if not ( fit_range[0] == None ):
            in_range = self.x > self.fit_range[0]
            self.x     = self.x[in_range]
            self.y     = self.y[in_range]
            self.x_err = self.x_err[in_range]
            self.y_err = self.y_err[in_range]
        if not ( fit_range[1] == None ):
            in_range = self.x < self.fit_range[1]
            self.x     = self.x[in_range]
            self.y     = self.y[in_range]
            self.x_err = self.x_err[in_range]
            self.y_err = self.y_err[in_range]

    # least squares fit
    def least_squares_fit(self):
        print("least squares fitting ...")
        coeff, covar = curve_fit(linear, self.x, self.y, sigma=self.y_err, p0=[1, 1])
        a, c  = coeff
        a_err = covar[0][0]
        c_err = covar[1][1]
        self.a['ls'] = [a,a_err]
        self.c['ls'] = [c,c_err]

    # maximum likelihood fit
    def maximum_likelihood_fit(self):
        print("maximum likelihood fitting ...")
        result = op.minimize(neg_lnlikelihood, [self.a['ls'][0], self.c['ls'][0]], args=(self.x, self.y, self.x_err, self.y_err))
        self.a['ml'], self.c['ml'] = result["x"]

    # run MCMC fit
    def MCMC_fit(self):
        print("Bayesian MCMC fitting ...")
        # initial values for walkers from least squares and maximum likelihood fits
        init_pos = [np.array([self.a['ml'], self.c['ml']]) +1e-2*np.random.randn(self.ndim) for i in range(self.nwalkers)]
        # run MCMC
        self.sampler = emcee.EnsembleSampler(self.nwalkers, self.ndim, lnprobability, args=(self.x, self.y, self.x_err, self.y_err))
        self.sampler.run_mcmc(init_pos, self.nsteps)
        self.samples = self.sampler.chain[:, 50:, :].reshape((-1, self.ndim))
        a_percentiles, c_percentiles = np.percentile(self.samples, [16, 50, 84], axis=0).T
        self.a['MCMC'] = [a_percentiles[1], a_percentiles[2]-a_percentiles[1], a_percentiles[1]-a_percentiles[0]]
        self.c['MCMC'] = [c_percentiles[1], c_percentiles[2]-c_percentiles[1], c_percentiles[1]-c_percentiles[0]]
        self.a['perc'] = [a_percentiles[0], a_percentiles[1], a_percentiles[2]]
        self.c['perc'] = [c_percentiles[0], c_percentiles[1], c_percentiles[2]]

    # plot walkers
    def plot_walkers(self):
        print("plotting walkers ...")
        fig,ax = plt.subplots(2, sharex=True)
        for w in np.arange(self.nwalkers):
            ax[0].plot(self.sampler.chain[w,:,0], color='k', linestyle='-', alpha=0.1)
            ax[1].plot(self.sampler.chain[w,:,1], color='k', linestyle='-', alpha=0.1)
        ax[1].set_xlabel('step')
        ax[0].set_ylabel('slope a')
        ax[1].set_ylabel('intercept c')
        ax[1].set_xlim(0,self.nsteps)
        mkdir(escape_filename(self.savepath))
        fig.savefig(os.path.join(self.savepath, self.source+'.'+self.line+'.walkers.png'), dpi=300, bbox_inches='tight')

    # corner plot
    def corner_plot(self):
        print("plotting parameter distributions ...")
        fig = corner.corner(self.samples, labels=['a', 'c'], quantiles=[0.16, 0.5, 0.84], show_titles=True)
        mkdir(escape_filename(self.savepath))
        fig.savefig(os.path.join(self.savepath, self.source+'.'+self.line+'.corner.png'), dpi=300, bbox_inches='tight')

    # plot x - y relation
    def plot_x_y(self):
        print("plotting x - y relation ...")
        fig,ax = plt.subplots(figsize=self.figsize)
        ax.scatter(np.power(10,self.x), np.power(10,self.y), marker='.', color='k', label='data', s=1, alpha=0.5, zorder=0)
        x = np.logspace(np.min(self.x)-np.log10(1.5), np.max(self.x)+np.log10(1.5), 100)

        # plot most likely
        ax.plot(x, powlaw(x,self.a['MCMC'][0],np.power(10,self.c['MCMC'][0])), lw=1, color="r", zorder=2, label='50 percentile fit')

        # plot random samples
        a, c = self.samples[np.random.randint(len(self.samples), size=1)][0]
        b = np.power(10,c)
        ax.plot(x, powlaw(x,a,b), color="k", alpha=0.1, zorder=2, label='random samples')
        for a, c in self.samples[np.random.randint(len(self.samples), size=10)]:
            b = np.power(10,c)
            ax.plot(x, powlaw(x,a,b), color="k", lw=1, alpha=0.1, zorder=1)

        ax.legend()
        ax.set_xlabel(self.xlabel)
        ax.set_ylabel(self.ylabel)
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlim([0.75*np.power(10,np.min(self.x)),1.5*np.power(10,np.max(self.x))])
        ax.set_ylim([0.75*np.power(10,np.min(self.y)),  1.5*np.power(10,np.max(self.y))])
        mkdir(escape_filename(self.savepath))
        fig.savefig(os.path.join(self.savepath, self.source+'.'+self.line+'.xy.png'), dpi=300, bbox_inches='tight')

    def list_results(self):
        print("\n\n")
        print(self.source+" "+self.line)
        print("  "+'{:>10}{:>10}{:>10}'.format('least sq','max like','MCMC'))
        print("a "+'{:10.6f}{:10.6f}{:10.6f}'.format(self.a['ls'][0], self.a['ml'], self.a['MCMC'][0]))
        print("a-"+'{:10.6f}{:>10}{:10.6f}'.format(  self.a['ls'][1], "",           self.a['MCMC'][1]))
        print("a+"+'{:10.6f}{:>10}{:10.6f}'.format(  self.a['ls'][1], "",           self.a['MCMC'][2]))
        print("c "+'{:10.6f}{:10.6f}{:10.6f}'.format(self.c['ls'][0], self.c['ml'], self.c['MCMC'][0]))
        print("c-"+'{:10.6f}{:>10}{:10.6f}'.format(  self.c['ls'][1], "",           self.c['MCMC'][1]))
        print("c+"+'{:10.6f}{:>10}{:10.6f}'.format(  self.c['ls'][1], "",           self.c['MCMC'][2]))
        print("\n\n")

    def get_y10(self):
        """Calculate y for a scale of 10. This is more meaningful than the intercept (scale=1)."""
        # MCMC
        a  = self.a['perc'][1]
        am = self.a['perc'][0]
        ap = self.a['perc'][2]
        c  = self.c['perc'][1]
        cm = self.c['perc'][0]
        cp = self.c['perc'][2]
        # least squares
        a_ls     = self.a['ls'][0]
        a_ls_err = self.a['ls'][1]
        c_ls     = self.c['ls'][0]
        c_ls_err = self.c['ls'][1]
        y10_ls     = powlaw(10, a_ls, np.power(10,c_ls))
        y10_ls_err = np.log(10)*np.power(10,a_ls+c_ls) *np.sqrt(a_ls_err**2+c_ls_err**2)

        # best fit sigma10
        self.y10 = {'MCMC': [powlaw(10, a, np.power(10,c)), powlaw(10, am, np.power(10,cm)), powlaw(10, ap, np.power(10,cp))],
                    'ls':   [y10_ls, y10_ls_err]}

        # distribution of sigma10
        self.y10s = {'MCMC': [powlaw(10, a, np.power(10,c)) for a, c in self.samples[np.random.randint(len(self.samples), size=100)]]}


####################################################################################################
# fit and plot meta function
####################################################################################################

def fit_MCMC(log_x, log_y, log_x_err, log_y_err, source, line, savepath, xlabel=None, ylabel=None, plotcolor='r', fit_range=[None,None]):
    MCMC = MCMC_fit(log_x, log_y, log_x_err, log_y_err)
    MCMC.savepath = savepath
    MCMC.source   = source
    MCMC.line     = line
    MCMC.plotcolor = plotcolor
    MCMC.xlabel   = xlabel
    MCMC.ylabel   = ylabel

    MCMC.mask_bad_values()
    MCMC.restrict_range(fit_range)
    MCMC.least_squares_fit()
    MCMC.maximum_likelihood_fit()
    MCMC.MCMC_fit()
    MCMC.plot_walkers()
    MCMC.corner_plot()
    MCMC.plot_x_y()
    MCMC.get_y10()
    MCMC.fit = True
    MCMC.list_results()

    return MCMC


####################################################################################################
# print formatted slopes and intercepts
####################################################################################################

def print_formatted(MCMC_list, datatype):
    print(datatype)
    print('{:>8} & {:>10} & {:>6} & {:>6} & {:>6} & {:>6} & {:>6} & {:>6} & {:>8} & {:>8} & {:>8}'.format('source','line','a_16','a_50','a_84','c_16','c_50','c_84','y10','y10 -','y10 +'))
    for m in MCMC_list:
        try:
            print('{:>8} & {:>10} & {:6.2f} & {:6.2f} & {:6.2f} & {:6.2f} & {:6.2f} & {:6.2f} & {:8.1f} & {:8.1f} & {:8.1f}'.format(m.source, m.line, m.a['perc'][0], m.a['perc'][1], m.a['perc'][2], m.c['perc'][0], m.c['perc'][1], m.c['perc'][2], m.y10[0], m.y10[1], m.y10[2]))
        except:
            print('{:>8} & {:>10} & {:<10}'.format(m.source, m.line, 'no fit'))
            
def mkdir(path):
    """mkdir
    Parameters
    ----------
    path : str
        Path to directory to create
    """
    import os
    if ' ' in path:
        raise Exception("Path contains spaces! This will most probably not create the directory you want!")
    if not os.path.exists(path):
        os.system('mkdir -p '+path)
        print("Created "+path)
        
def escape_filename(str):
    """Escape the most often used characters in a string to be used as a file name.
    Parameters
    ----------
    str : str
        Input string to be escaped.
    Returns
    -------
    str
        String with escaped '(', ')'; replaced ' ' by '_' and removed '$'.
        
    """
    str = str.replace('(',r'\(')
    str = str.replace(')',r'\)')
    str = str.replace(' ',r'_')
    str = str.replace('$','')
    return str


def filter_window(x,y,window_min,window_max):
    """select values in the window"""
    x_window = x[(window_min<x) & (x<window_max)]
    y_window = y[(window_min<x) & (x<window_max)]
    return x_window,y_window


def crossmatch(*args):
    """Crossmatch lists for non-finite values.
    Parameters
    ----------
    x : list
    y : list
    ...
    Returns
    -------
    list, list
        Lists in input order with the non-finite (infinite and NaN) values removed from the list and
        also the corresponding element of the other list.
    """
    import numpy as np

    lists = []
    for list in args:
        lists.append( np.array(list) )

    selection = np.isfinite(lists[0])
    for list in lists[1:]:
        selection = selection & np.isfinite(list)

    matched_lists = []
    for list in lists:
        matched_lists.append( list[selection] )

    return matched_lists
def get_binned_percentiles(x_measure, y_measure, x_step, xdata, ydata):

    from astropy.table import Table
    
    

    def filter_window(x,y,window_min,window_max):
        """select values in the window"""
        x_window = x[(window_min<x) & (x<window_max)]
        y_window = y[(window_min<x) & (x<window_max)]
        return x_window,y_window


    # binned percentiles table
    bin_perc = Table(names = ['window min','window center','window max','x 16th','x median','x 84th','y 16th','y median','y 84th'],
                     meta = {'x': x_measure, 'y': y_measure, 'x_step': x_step}
                    )

    x_matched, y_matched = crossmatch(xdata,ydata)

    x_min = np.nanmin(x_matched)
    x_max = np.nanmax(x_matched)
    y_min = np.nanmin(y_matched)
    y_max = np.nanmax(y_matched)

    for log_window_cen in np.arange(np.log10(x_min), np.log10(x_max)+x_step, x_step):
        log_window_min = log_window_cen-x_step/2.
        log_window_max = log_window_cen+x_step/2.
        window_min = np.power(10,log_window_min)
        window_cen = np.power(10,log_window_cen)
        window_max = np.power(10,log_window_max)
        x_window,y_window = filter_window(x_matched,y_matched,window_min,window_max)

        # percentiles only make sense when at least two measurements are available
        if len(x_window)>1:
            x_perc = np.percentile(x_window, [16,50,84])
            y_perc = np.percentile(y_window, [16,50,84])
            bin_perc.add_row( flatten([window_min,window_cen,window_max,x_perc,y_perc]) )

    return bin_perc

    
    
def flatten(x):
    """Flatten list of lists.
    Parameters
    ----------
    x : list,tuple
        List of lists of lists of ...
        Can contain a mixture of lists and values.
    Returns
    -------
    list
        Flattened list.
    """
    import collections
    if isinstance(x, collections.Iterable):
        return [a for i in x for a in flatten(i)]
    else:
        return [x]




C:\Users\b347m182\Anaconda new\python.exe
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1
reproject 0.9.1
spectral_cube 0.6.1.dev247+ge29d254
C:\Users\b347m182\Anaconda new\lib\site-packages\spectral_cube\__init__.py
C:\Users\b347m182\Anaconda new\lib\site-packages\astrodendro\__init__.py
1.23.0 Numpy


# Part 6: Reprojection

In [ ]:
#Just Using CO 3-2 right now

files = ["12CO_GC_359-000_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read(files[0])  
Cube_Name = files[0]+"Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write(Cube_Name,overwrite=True)
print(4)
print("done")

In [ ]:
#Just Using CO 3-2 right now

files = ["12CO_GC_359-000_mosaic.fit"+"Header_Fix.fits"]

#files=["Pre-Smoothed.fits"]
      
#FOV = [400,800]#pc
FOV = [70,360]#pc
gal="GC"
Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_CMZ_CO_J3_2_4.3_start'
Side="RH"
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
#Beam_Sizes = np.linspace(Smallest_beam,Smallest_beam*iterations, np.diff(Smallest_beam,Smallest_beam*iterations)/iterations)
iter_factor = 1/5
Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"
    
    
for kl in range(5,6):
    Prime_Beam = Min_res*((kl*iter_factor+1))#Beam_Sizes[kl]
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        ovs=5
        FOVp=[200,800]
    else:
        ovs=3
        FOVp=FOV
    #Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high)
    

    print(kl)
    Cube_Name = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+Side+'pc_NEW.fits'
    sc = SpectralCube.read(files[0])  


    Nres=4.3*u.pc
    dist=8.178*10**3#pc
    res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
    print(res,"A")



    sc = sc[:,:,:]# Make a subcube

    
    #Put in the right system
    sc.allow_huge_operations=True
    sc_kms = sc#.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s

    vel,RA,Dec = sc_kms.world[:,0,0]




    try:


        sc = sc_kms.spectral_slab(-325. *u.km / u.s, 326. *u.km / u.s)  # Crop out velocities we don't care about  

        del sc_kms

        HI = sc.header
        Nres=Prime_Beam
        if(gal=="NGC253"):
            dist=3.5*10**6
        if(gal=="GC"):
            dist=8.178*10**3#pc
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc

        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)


        sc.allow_huge_operations=True







        #Now that we have a circular beam, this can be done easily:

        if(gal == 'GC'):

            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            #center = SkyCoord('0359d56m39.24s', '-00d02m46.176s', frame='galactic')
            center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
            center_ra_pix,center_dec_pix = int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])

            print(center_dec_pix,center_ra_pix)

            print((sc.wcs[:][:][0].pixel_to_world(center_dec_pix,center_ra_pix)),center_dec_pix,center_ra_pix)
            PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*8.178*10**3))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*8.178*10**3))/2]


            pixels = np.zeros(np.shape(sc))           
            for lmi in range(np.shape(sc)[0]):
                for lmj in range(np.shape(sc)[1]):
                    for lmk in range(np.shape(sc)[2]):

                        up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                        side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                        if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                            #print(lmj,lmk,np.shape(sc))
                            pixels[lmi][lmj][lmk] = 1#good
                    #print("Up pix",up_pixels,"Up pix fov",PixFov[0],"side pix",side_pixels,"side pix fov",PixFov[1],"lmj (dec)",lmj,"lmk (ra)",lmk,"center dec pix",center_dec_pix,"center ra pix",center_ra_pix,cdelt_x_pc,cdelt_y_pc)

            bp = np.where(pixels!=1)
            #print(pixels)

            #Mask teh pixels outside the fov
            scCopy = sc.hdu
            #sc.hdu.data[bp]=9999#np.nan
            scCopy.data[bp]=np.nan
            sc = SpectralCube.read(scCopy)
            del scCopy

        sc.allow_huge_operations=True

        
        try:
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
        except:
            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            if(cdelt_x>cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_y
            elif(cdelt_x<cdelt_y):
                majorBase=cdelt_y
                minorBase=cdelt_x
            elif(cdelt_x==cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_x
            BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

            sc = sc.with_beam(BaseBeam)

            beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
            print(BaseBeam,beam,"C")
            sc.allow_huge_operations=True
            print(sc.shape,np.nanmax(sc.hdu.data),'EA')
            #Requires me to edit convolve.py and set allow_huge =True
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
            print(np.shape(sc),"B")












        reheader = copy.deepcopy(sc.hdu.header)


        if(gal=="NGC253"):
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 
            if sc.header['cdelt1']>0:
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                pix_x    = -1.*(res/ovs).to(u.degree).value
                origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value

            if sc.header['cdelt2']>0:
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[0].to(u.degree).value
            else:
                pix_y    = -1.*(res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[1].to(u.degree).value

            if gal=='NGC253':
                npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
                npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)

        elif gal=='GC':
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            if sc.header['cdelt1']>0:
                #pix_x    = sc.header['cdelt1']/5#(res/5.).to(u.degree).value*10
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = 358.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = .55/3*res.to(u.degree).value
                    origin_x = 358.6#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                    
                #origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_x    = sc.header['cdelt1']/5#-1.*(res/5.).to(u.degree).value*10
                pix_x    = -(res/ovs).to(u.degree).value
                origin_x = 1.5#(sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                #origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = -.55/3*res.to(u.degree).value
                    origin_x = .9

            if sc.header['cdelt2']>0:
                #pix_y    = sc.header['cdelt2']/5#(res/5.).to(u.degree).value*10
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = -.5#sc.latitude_extrema[0].to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = .55/3*res.to(u.degree).value
                    origin_y = -0.6#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_y    = sc.header['cdelt2']/5#-1.*(res/5.).to(u.degree).value*10
                pix_y    = -(res/ovs).to(u.degree).value
                origin_y = .5#sc.latitude_extrema[1].to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = -.55/3*res.to(u.degree).value
                    origin_y = 0.6#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value



            if gal=='GC':
                #manually put in the size to correct for the 360->0 difference and just because it doesnt seem to work
                print("LA",((sc.longitude_extrema[0])),(sc.longitude_extrema[1]-360*u.degree),np.abs(pix_x))
                print("MA",sc.latitude_extrema,np.abs(pix_y))
                #npix_x   =int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
                #npix_y   =int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                npix_x   =int(2.5/np.abs(pix_x))
                npix_y   =int(1.2/np.abs(pix_y))
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    npix_x   =int(2.3/np.abs(pix_x))
                    npix_y   =int(1.2/np.abs(pix_y))

                print(npix_x,npix_y)


        #Correct the header to the expected pixels for the new res

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y

        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y

        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y

        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        
        reheader['CTYPE1'] = "GLON-SIN"
        reheader['CTYPE2'] = "GLAT-SIN"
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']#Dont need these anymore, maybe?
            
            del reheader['LBOUND1']
            del reheader['LBOUND2']
            del reheader['LBOUND3']
            del reheader.cards['LBOUND1']
            del reheader.cards['LBOUND2']
            del reheader.cards['LBOUND3']
            
            reheader['LBOUND1']=0
            reheader['LBOUND2']=0
            reheader['LBOUND3']=0
            
            print("12312412")
        except Exception as e:
            print(e)
            
            print("Failed")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)
        
        # regrid cube to target pixel size
        
        
        
        #Cube_Name_Load_HCN = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+'_CMZ_HCN_J1_0_4.3_start'+str(FOV[0])+"x"+str(FOV[1])+'pc_'+str(3.3)+'_vel_res.fits'
        #del reheader
        #reheader = SpectralCube.read(Cube_Name_Load_HCN).hdu.header
        #reheader["CTYPE3"] = "VRAD"
        
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        del sc
        #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
        new = SpectralCube(data=sc_K_kms.hdu.data,wcs =WCS(sc_K_kms.header),header=sc_K_kms.header,mask=sc_K_kms.mask)
        new.allow_huge_operations=True
        new = new*sc_K_kms[0][0][0].unit
        #do this because scs dont like being modified
        del sc_K_kms
        sc_K_kms = new
        del new

        print(np.nanmax(sc_K_kms),np.shape(sc_K_kms))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
        

        NGC_CO_J3_2 = sc_K_kms
        del sc_K_kms
        NGC_CO_J3_2.allow_huge_operations=True

        #do this again to crop the extra pixels off

        sc=NGC_CO_J3_2

        


        #Write the intermediary cubes that will then be spliced together
        del NGC_CO_J3_2
        

        

        fwhm_factor = np.sqrt(8*np.log(2))

        vel,RA,Dec =sc.world[:,0,0]
        for km in range(5,6):
            scWsc_copy = sc
            vel_prime = min_vel*((km*iter_factor+1))
            if(km==5):
                vel_prime=2.5
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            if(gal=="GC"):
                vel = np.arange(-300,301,vel_prime)*u.km/u.s    
            else:
                vel = np.arange(0,501,vel_prime)*u.km/u.s    

            G_width = np.sqrt(2.5**2-1**2)
            if(km==5):
                #G_width = 4
                G_width = np.sqrt(2.5**2-1**2)
                if(Match_to_HCO):
                    G_width = np.sqrt(3.3**2-1**2)
            if(km==5 and Smoothe_4):
                G_width = 4
            if(km==5 and Smoothe_2_5):
                G_width = 2.5/1#G_width = 2.5/1
                
            scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
            scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)
            scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range  



            scWsc_copy.write(Cube_Name_Save,overwrite=True)
            gc.collect()
            #scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32.fits",overwrite=True)
        del scWsc_copy
        
        del sc
        
        gc.collect()######################################################################
    except Exception as e:
        print(e)
        print(kl)
        print("Failed")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)


      
    
print("done")      

In [ ]:
#Just Using CO 3-2 right now

files = ["12CO_GC_001-002_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read(files[0])  
Cube_Name = files[0]+"Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write(Cube_Name,overwrite=True)
print(4)
print("done")

In [ ]:
#print(np.shape(bp))
#np.savetxt("bp.txt",np.array(bp[1][999999:9999999],dtype=int))
#np.savetxt("bp2.txt",np.array(bp[2][0:99999],dtype=int))
#Just Using CO 3-2 right now
#files = ["12CO_GC_359-000_mosaic.fit"]
files = ["12CO_GC_001-002_mosaic.fit"]
files = [files[0]+"Header_Fix.fits"]
      
#files=["Pre-Smoothed.fits"]
      
#FOV = [400,800]#pc
FOV = [70,360]#pc
gal="GC"
Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_CMZ_CO_J3_2_4.3_start'
Side="LH"
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
#Beam_Sizes = np.linspace(Smallest_beam,Smallest_beam*iterations, np.diff(Smallest_beam,Smallest_beam*iterations)/iterations)
iter_factor = 1/5
Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"
for kl in range(5,6):
    Prime_Beam = Min_res*((kl*iter_factor+1))#Beam_Sizes[kl]
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        ovs=5
        FOVp=[200,800]
    else:
        ovs=3
        FOVp=FOV
    #Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high)
    

    print(kl)
    Cube_Name = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+Side+'pc_NEW.fits'
    sc = SpectralCube.read(files[0])  


    Nres=4.3*u.pc
    dist=8.178*10**3#pc
    res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
    print(res,"A")



    sc = sc[:,:,:]# Make a subcube

    
    #Put in the right system
    sc.allow_huge_operations=True
    sc_kms = sc#.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s

    vel,RA,Dec = sc_kms.world[:,0,0]




    try:


        sc = sc_kms.spectral_slab(-325. *u.km / u.s, 326. *u.km / u.s)  # Crop out velocities we don't care about  

        del sc_kms

        HI = sc.header
        Nres=Prime_Beam
        if(gal=="NGC253"):
            dist=3.5*10**6
        if(gal=="GC"):
            dist=8.178*10**3#pc
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc

        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)


        sc.allow_huge_operations=True







        #Now that we have a circular beam, this can be done easily:

        if(gal == 'GC'):

            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            #center = SkyCoord('0359d56m39.24s', '-00d02m46.176s', frame='galactic')
            center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
            center_ra_pix,center_dec_pix = int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])

            print(center_dec_pix,center_ra_pix)

            print((sc.wcs[:][:][0].pixel_to_world(center_dec_pix,center_ra_pix)),center_dec_pix,center_ra_pix)
            PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*8.178*10**3))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*8.178*10**3))/2]


            pixels = np.zeros(np.shape(sc))           
            for lmi in range(np.shape(sc)[0]):
                for lmj in range(np.shape(sc)[1]):
                    for lmk in range(np.shape(sc)[2]):

                        up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                        side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                        if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                            #print(lmj,lmk,np.shape(sc))
                            pixels[lmi][lmj][lmk] = 1#good
                    #print("Up pix",up_pixels,"Up pix fov",PixFov[0],"side pix",side_pixels,"side pix fov",PixFov[1],"lmj (dec)",lmj,"lmk (ra)",lmk,"center dec pix",center_dec_pix,"center ra pix",center_ra_pix,cdelt_x_pc,cdelt_y_pc)

            bp = np.where(pixels!=1)
            #print(pixels)

            #Mask teh pixels outside the fov
            scCopy = sc.hdu
            #sc.hdu.data[bp]=9999#np.nan
            scCopy.data[bp]=np.nan
            sc = SpectralCube.read(scCopy)
            del scCopy

        sc.allow_huge_operations=True

        
        try:
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
        except:
            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            if(cdelt_x>cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_y
            elif(cdelt_x<cdelt_y):
                majorBase=cdelt_y
                minorBase=cdelt_x
            elif(cdelt_x==cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_x
            BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

            sc = sc.with_beam(BaseBeam)

            beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
            print(BaseBeam,beam,"C")
            sc.allow_huge_operations=True
            print(sc.shape,np.nanmax(sc.hdu.data),'EA')
            #Requires me to edit convolve.py and set allow_huge =True
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
            print(np.shape(sc),"B")












        reheader = copy.deepcopy(sc.hdu.header)


        if(gal=="NGC253"):
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 
            if sc.header['cdelt1']>0:
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                pix_x    = -1.*(res/ovs).to(u.degree).value
                origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value

            if sc.header['cdelt2']>0:
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[0].to(u.degree).value
            else:
                pix_y    = -1.*(res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[1].to(u.degree).value

            if gal=='NGC253':
                npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
                npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)

        elif gal=='GC':
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            if sc.header['cdelt1']>0:
                #pix_x    = sc.header['cdelt1']/5#(res/5.).to(u.degree).value*10
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = 358.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = .55/3*res.to(u.degree).value
                    origin_x = 357.2#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                    
                #origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_x    = sc.header['cdelt1']/5#-1.*(res/5.).to(u.degree).value*10
                pix_x    = -(res/ovs).to(u.degree).value
                origin_x = 1.5#(sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                #origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = -.55/3*res.to(u.degree).value
                    origin_x = 2.8

            if sc.header['cdelt2']>0:
                #pix_y    = sc.header['cdelt2']/5#(res/5.).to(u.degree).value*10
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = -.5#sc.latitude_extrema[0].to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = .55/3*res.to(u.degree).value
                    origin_y = -0.55#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_y    = sc.header['cdelt2']/5#-1.*(res/5.).to(u.degree).value*10
                pix_y    = -(res/ovs).to(u.degree).value
                origin_y = .5#sc.latitude_extrema[1].to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = -.55/3*res.to(u.degree).value
                    origin_y = 0.55#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value



            if gal=='GC':
                #manually put in the size to correct for the 360->0 difference and just because it doesnt seem to work
                print("LA",((sc.longitude_extrema[0])),(sc.longitude_extrema[1]-360*u.degree),np.abs(pix_x))
                print("MA",sc.latitude_extrema,np.abs(pix_y))
                #npix_x   =int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
                #npix_y   =int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                npix_x   =int(3/np.abs(pix_x))
                npix_y   =int(1.2/np.abs(pix_y))
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    npix_x   =int(2.5/np.abs(pix_x))
                    npix_y   =int(1.1/np.abs(pix_y))

                print(npix_x,npix_y)


        #Correct the header to the expected pixels for the new res

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y

        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y

        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y

        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        
        reheader['CTYPE1'] = "GLON-SIN"
        reheader['CTYPE2'] = "GLAT-SIN"
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']#Dont need these anymore, maybe?
            
            del reheader['LBOUND1']
            del reheader['LBOUND2']
            del reheader['LBOUND3']
            del reheader.cards['LBOUND1']
            del reheader.cards['LBOUND2']
            del reheader.cards['LBOUND3']
            
            reheader['LBOUND1']=0
            reheader['LBOUND2']=0
            reheader['LBOUND3']=0
            
            print("12312412")
        except Exception as e:
            print(e)
            
            print("Failed")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)
        
        # regrid cube to target pixel size
        
        
        
        #Cube_Name_Load_HCN = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+'_CMZ_HCN_J1_0_4.3_start'+str(FOV[0])+"x"+str(FOV[1])+'pc_'+str(3.3)+'_vel_res.fits'
        #del reheader
        #reheader = SpectralCube.read(Cube_Name_Load_HCN).hdu.header
        #reheader["CTYPE3"] = "VRAD"
        
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        del sc
        #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
        new = SpectralCube(data=sc_K_kms.hdu.data,wcs =WCS(sc_K_kms.header),header=sc_K_kms.header,mask=sc_K_kms.mask)
        new.allow_huge_operations=True
        new = new*sc_K_kms[0][0][0].unit
        #do this because scs dont like being modified
        del sc_K_kms
        sc_K_kms = new
        del new

        print(np.nanmax(sc_K_kms),np.shape(sc_K_kms))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
        

        NGC_CO_J3_2 = sc_K_kms
        del sc_K_kms
        NGC_CO_J3_2.allow_huge_operations=True

        #do this again to crop the extra pixels off

        sc=NGC_CO_J3_2

        sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc


        #Write the intermediary cubes that will then be spliced together
        del NGC_CO_J3_2
        

        

        fwhm_factor = np.sqrt(8*np.log(2))

        vel,RA,Dec =sc.world[:,0,0]
        for km in range(5,6):
            scWsc_copy = sc
            vel_prime = min_vel*((km*iter_factor+1))
            if(km==5):
                vel_prime=2.5
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            if(gal=="GC"):
                vel = np.arange(-300,301,vel_prime)*u.km/u.s    
            else:
                vel = np.arange(0,501,vel_prime)*u.km/u.s    

            G_width = np.sqrt(2.5**2-1**2)
            if(km==5):
                #G_width = 4
                G_width = np.sqrt(2.5**2-1**2)
                if(Match_to_HCO):
                    G_width = np.sqrt(3.3**2-1**2)
            if(km==5 and Smoothe_4):
                G_width = 4
            if(km==5 and Smoothe_2_5):
                G_width = 2.5/1#G_width = 2.5/1
                
            scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
            scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)
            scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range 



            scWsc_copy.write(Cube_Name_Save,overwrite=True)
            gc.collect()
            #scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32.fits",overwrite=True)
        del scWsc_copy
        
        del sc
        
        gc.collect()######################################################################
    except Exception as e:
        print(e)
        print(kl)
        print("Failed")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)


      
    
print("done")   

In [ ]:
#Just Using CO 3-2 right now

files = ["12CO_GC_357-358_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read(files[0])  
Cube_Name = files[0]+"Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write(Cube_Name,overwrite=True)
print(4)
print("done")

In [ ]:
#print(np.shape(bp))
#np.savetxt("bp.txt",np.array(bp[1][999999:9999999],dtype=int))
#np.savetxt("bp2.txt",np.array(bp[2][0:99999],dtype=int))
#Just Using CO 3-2 right now
#files = ["12CO_GC_359-000_mosaic.fit"]
files = ["12CO_GC_357-358_mosaic.fit"]
files = [files[0]+"Header_Fix.fits"]
      

#files=["Pre-Smoothed.fits"]
      
#FOV = [400,800]#pc
FOV = [70,360]#pc
gal="GC"
Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_CMZ_CO_J3_2_4.3_start'
Side="FRH"
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
#Beam_Sizes = np.linspace(Smallest_beam,Smallest_beam*iterations, np.diff(Smallest_beam,Smallest_beam*iterations)/iterations)
iter_factor = 1/5
Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"
for kl in range(5,6):
    Prime_Beam = Min_res*((kl*iter_factor+1))#Beam_Sizes[kl]
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        ovs=5
        FOVp=[200,800]
    else:
        ovs=3
        FOVp=FOV
    #Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high)
    

    print(kl)
    Cube_Name = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+Side+'pc_NEW.fits'
    sc = SpectralCube.read(files[0])  


    Nres=4.3*u.pc
    dist=8.178*10**3#pc
    res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
    print(res,"A")



    sc = sc[:,:,:]# Make a subcube

    
    #Put in the right system
    sc.allow_huge_operations=True
    sc_kms = sc#.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s

    vel,RA,Dec = sc_kms.world[:,0,0]




    try:


        sc = sc_kms.spectral_slab(-325. *u.km / u.s, 326. *u.km / u.s)  # Crop out velocities we don't care about  

        del sc_kms

        HI = sc.header
        Nres=Prime_Beam
        if(gal=="NGC253"):
            dist=3.5*10**6
        if(gal=="GC"):
            dist=8.178*10**3#pc
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc

        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)


        sc.allow_huge_operations=True







        #Now that we have a circular beam, this can be done easily:

        if(gal == 'GC'):

            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            #center = SkyCoord('0359d56m39.24s', '-00d02m46.176s', frame='galactic')
            center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
            center_ra_pix,center_dec_pix = int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])

            print(center_dec_pix,center_ra_pix)

            print((sc.wcs[:][:][0].pixel_to_world(center_dec_pix,center_ra_pix)),center_dec_pix,center_ra_pix)
            PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*8.178*10**3))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*8.178*10**3))/2]


            pixels = np.zeros(np.shape(sc))           
            for lmi in range(np.shape(sc)[0]):
                for lmj in range(np.shape(sc)[1]):
                    for lmk in range(np.shape(sc)[2]):

                        up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                        side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                        if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                            #print(lmj,lmk,np.shape(sc))
                            pixels[lmi][lmj][lmk] = 1#good
                    #print("Up pix",up_pixels,"Up pix fov",PixFov[0],"side pix",side_pixels,"side pix fov",PixFov[1],"lmj (dec)",lmj,"lmk (ra)",lmk,"center dec pix",center_dec_pix,"center ra pix",center_ra_pix,cdelt_x_pc,cdelt_y_pc)

            bp = np.where(pixels!=1)
            #print(pixels)

            #Mask teh pixels outside the fov
            scCopy = sc.hdu
            #sc.hdu.data[bp]=9999#np.nan
            scCopy.data[bp]=np.nan
            sc = SpectralCube.read(scCopy)
            del scCopy

        sc.allow_huge_operations=True

        
        try:
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
        except:
            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            if(cdelt_x>cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_y
            elif(cdelt_x<cdelt_y):
                majorBase=cdelt_y
                minorBase=cdelt_x
            elif(cdelt_x==cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_x
            BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

            sc = sc.with_beam(BaseBeam)

            beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
            print(BaseBeam,beam,"C")
            sc.allow_huge_operations=True
            print(sc.shape,np.nanmax(sc.hdu.data),'EA')
            #Requires me to edit convolve.py and set allow_huge =True
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
            print(np.shape(sc),"B")












        reheader = copy.deepcopy(sc.hdu.header)


        if(gal=="NGC253"):
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 
            if sc.header['cdelt1']>0:
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                pix_x    = -1.*(res/ovs).to(u.degree).value
                origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value

            if sc.header['cdelt2']>0:
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[0].to(u.degree).value
            else:
                pix_y    = -1.*(res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[1].to(u.degree).value

            if gal=='NGC253':
                npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
                npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)

        elif gal=='GC':
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            if sc.header['cdelt1']>0:
                #pix_x    = sc.header['cdelt1']/5#(res/5.).to(u.degree).value*10
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = 358.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = .55/3*res.to(u.degree).value
                    origin_x = 357#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                    
                #origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_x    = sc.header['cdelt1']/5#-1.*(res/5.).to(u.degree).value*10
                pix_x    = -(res/ovs).to(u.degree).value
                origin_x = 1.5#(sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                #origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = -.55/3*res.to(u.degree).value
                    origin_x = 359

            if sc.header['cdelt2']>0:
                #pix_y    = sc.header['cdelt2']/5#(res/5.).to(u.degree).value*10
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = -.5#sc.latitude_extrema[0].to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = .55/3*res.to(u.degree).value
                    origin_y = -0.55#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_y    = sc.header['cdelt2']/5#-1.*(res/5.).to(u.degree).value*10
                pix_y    = -(res/ovs).to(u.degree).value
                origin_y = .5#sc.latitude_extrema[1].to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = -.55/3*res.to(u.degree).value/3*res
                    origin_y = 0.55#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value



            if gal=='GC':
                #manually put in the size to correct for the 360->0 difference and just because it doesnt seem to work
                print("LA",((sc.longitude_extrema[0])),(sc.longitude_extrema[1]-360*u.degree),np.abs(pix_x))
                print("MA",sc.latitude_extrema,np.abs(pix_y))
                #npix_x   =int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
                #npix_y   =int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                npix_x   =int(3/np.abs(pix_x))
                npix_y   =int(1.2/np.abs(pix_y))
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    npix_x   =int(2/np.abs(pix_x))
                    npix_y   =int(1.1/np.abs(pix_y))

                print(npix_x,npix_y)


        #Correct the header to the expected pixels for the new res

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y

        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y

        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y

        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        
        reheader['CTYPE1'] = "GLON-SIN"
        reheader['CTYPE2'] = "GLAT-SIN"
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']#Dont need these anymore, maybe?
            
            del reheader['LBOUND1']
            del reheader['LBOUND2']
            del reheader['LBOUND3']
            del reheader.cards['LBOUND1']
            del reheader.cards['LBOUND2']
            del reheader.cards['LBOUND3']
            
            reheader['LBOUND1']=0
            reheader['LBOUND2']=0
            reheader['LBOUND3']=0
            
            print("12312412")
        except Exception as e:
            print(e)
            
            print("Failed")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)
        
        # regrid cube to target pixel size
        
        
        
        #Cube_Name_Load_HCN = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+'_CMZ_HCN_J1_0_4.3_start'+str(FOV[0])+"x"+str(FOV[1])+'pc_'+str(3.3)+'_vel_res.fits'
        #del reheader
        #reheader = SpectralCube.read(Cube_Name_Load_HCN).hdu.header
        #reheader["CTYPE3"] = "VRAD"
        
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        del sc
        #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
        new = SpectralCube(data=sc_K_kms.hdu.data,wcs =WCS(sc_K_kms.header),header=sc_K_kms.header,mask=sc_K_kms.mask)
        new.allow_huge_operations=True
        new = new*sc_K_kms[0][0][0].unit
        #do this because scs dont like being modified
        del sc_K_kms
        sc_K_kms = new
        del new

        print(np.nanmax(sc_K_kms),np.shape(sc_K_kms))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
        

        NGC_CO_J3_2 = sc_K_kms
        del sc_K_kms
        NGC_CO_J3_2.allow_huge_operations=True

        #do this again to crop the extra pixels off

        sc=NGC_CO_J3_2

        sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc


        #Write the intermediary cubes that will then be spliced together
        del NGC_CO_J3_2
        

        

        fwhm_factor = np.sqrt(8*np.log(2))

        vel,RA,Dec =sc.world[:,0,0]
        for km in range(5,6):
            scWsc_copy = sc
            vel_prime = min_vel*((km*iter_factor+1))
            if(km==5):
                vel_prime=2.5
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            if(gal=="GC"):
                vel = np.arange(-300,301,vel_prime)*u.km/u.s    
            else:
                vel = np.arange(0,501,vel_prime)*u.km/u.s    

            G_width = np.sqrt(2.5**2-1**2)
            if(km==5):
                #G_width = 4
                G_width = np.sqrt(2.5**2-1**2)
                if(Match_to_HCO):
                    G_width = np.sqrt(3.3**2-1**2)
            if(km==5 and Smoothe_4):
                G_width = 4
            if(km==5 and Smoothe_2_5):
                G_width = 2.5/1#G_width = 2.5/1
                
            scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
            scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)
            scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range 



            scWsc_copy.write(Cube_Name_Save,overwrite=True)
            gc.collect()
            #scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32.fits",overwrite=True)
        del scWsc_copy
        
        del sc
        
        gc.collect()######################################################################
    except Exception as e:
        print(e)
        print(kl)
        print("Failed")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)


      
    
print("done")  

In [ ]:
#Just Using CO 3-2 right now

files = ["12CO_GC_003-005_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read(files[0])  
Cube_Name = files[0]+"Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write(Cube_Name,overwrite=True)
print(4)
print("done")

In [ ]:
#print(np.shape(bp))
#np.savetxt("bp.txt",np.array(bp[1][999999:9999999],dtype=int))
#np.savetxt("bp2.txt",np.array(bp[2][0:99999],dtype=int))
#Just Using CO 3-2 right now
#files = ["12CO_GC_359-000_mosaic.fit"]
files = ["12CO_GC_003-005_mosaic.fit"]
files = [files[0]+"Header_Fix.fits"]
      

#files=["Pre-Smoothed.fits"]
      
#FOV = [400,800]#pc
FOV = [70,360]#pc
gal="GC"
Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_CMZ_CO_J3_2_4.3_start'
Side="FLRH"
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
#Beam_Sizes = np.linspace(Smallest_beam,Smallest_beam*iterations, np.diff(Smallest_beam,Smallest_beam*iterations)/iterations)
iter_factor = 1/5
Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"
for kl in range(5,6):
    Prime_Beam = Min_res*((kl*iter_factor+1))#Beam_Sizes[kl]
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        ovs=5
        FOVp=[200,800]
    else:
        ovs=3
        FOVp=FOV
    #Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high)
    

    print(kl)
    Cube_Name = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+Side+'pc_NEW.fits'
    sc = SpectralCube.read(files[0])  


    Nres=4.3*u.pc
    dist=8.178*10**3#pc
    res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
    print(res,"A")



    sc = sc[:,:,:]# Make a subcube

    
    #Put in the right system
    sc.allow_huge_operations=True
    sc_kms = sc#.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s

    vel,RA,Dec = sc_kms.world[:,0,0]




    try:


        sc = sc_kms.spectral_slab(-325. *u.km / u.s, 326. *u.km / u.s)  # Crop out velocities we don't care about  

        del sc_kms

        HI = sc.header
        Nres=Prime_Beam
        if(gal=="NGC253"):
            dist=3.5*10**6
        if(gal=="GC"):
            dist=8.178*10**3#pc
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc

        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)


        sc.allow_huge_operations=True







        #Now that we have a circular beam, this can be done easily:

        if(gal == 'GC'):

            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            #center = SkyCoord('0359d56m39.24s', '-00d02m46.176s', frame='galactic')
            center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
            center_ra_pix,center_dec_pix = int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])

            print(center_dec_pix,center_ra_pix)

            print((sc.wcs[:][:][0].pixel_to_world(center_dec_pix,center_ra_pix)),center_dec_pix,center_ra_pix)
            PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*8.178*10**3))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*8.178*10**3))/2]


            pixels = np.zeros(np.shape(sc))           
            for lmi in range(np.shape(sc)[0]):
                for lmj in range(np.shape(sc)[1]):
                    for lmk in range(np.shape(sc)[2]):

                        up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                        side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                        if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                            #print(lmj,lmk,np.shape(sc))
                            pixels[lmi][lmj][lmk] = 1#good
                    #print("Up pix",up_pixels,"Up pix fov",PixFov[0],"side pix",side_pixels,"side pix fov",PixFov[1],"lmj (dec)",lmj,"lmk (ra)",lmk,"center dec pix",center_dec_pix,"center ra pix",center_ra_pix,cdelt_x_pc,cdelt_y_pc)

            bp = np.where(pixels!=1)
            #print(pixels)

            #Mask teh pixels outside the fov
            scCopy = sc.hdu
            #sc.hdu.data[bp]=9999#np.nan
            scCopy.data[bp]=np.nan
            sc = SpectralCube.read(scCopy)
            del scCopy

        sc.allow_huge_operations=True

        
        try:
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
        except:
            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            if(cdelt_x>cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_y
            elif(cdelt_x<cdelt_y):
                majorBase=cdelt_y
                minorBase=cdelt_x
            elif(cdelt_x==cdelt_y):
                majorBase=cdelt_x
                minorBase=cdelt_x
            BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

            sc = sc.with_beam(BaseBeam)

            beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
            print(BaseBeam,beam,"C")
            sc.allow_huge_operations=True
            print(sc.shape,np.nanmax(sc.hdu.data),'EA')
            #Requires me to edit convolve.py and set allow_huge =True
            sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
            print(np.shape(sc),"B")












        reheader = copy.deepcopy(sc.hdu.header)


        if(gal=="NGC253"):
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 
            if sc.header['cdelt1']>0:
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                pix_x    = -1.*(res/ovs).to(u.degree).value
                origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value

            if sc.header['cdelt2']>0:
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[0].to(u.degree).value
            else:
                pix_y    = -1.*(res/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[1].to(u.degree).value

            if gal=='NGC253':
                npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
                npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)

        elif gal=='GC':
            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            if sc.header['cdelt1']>0:
                #pix_x    = sc.header['cdelt1']/5#(res/5.).to(u.degree).value*10
                pix_x    = (res/ovs).to(u.degree).value
                origin_x = 358.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = .55/3*res.to(u.degree).value
                    origin_x = 2#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
                    
                #origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_x    = sc.header['cdelt1']/5#-1.*(res/5.).to(u.degree).value*10
                pix_x    = -(res/ovs).to(u.degree).value
                origin_x = 1.5#(sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                #origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_x    = -.55/3*res.to(u.degree).value
                    origin_x = 4

            if sc.header['cdelt2']>0:
                #pix_y    = sc.header['cdelt2']/5#(res/5.).to(u.degree).value*10
                pix_y    = (res/ovs).to(u.degree).value
                origin_y = -.5#sc.latitude_extrema[0].to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = .55/3*res.to(u.degree).value
                    origin_y = -0.55#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

            else:
                #pix_y    = sc.header['cdelt2']/5#-1.*(res/5.).to(u.degree).value*10
                pix_y    = -(res/ovs).to(u.degree).value
                origin_y = .5#sc.latitude_extrema[1].to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    #pix_y    = -.55/3*res.to(u.degree).value/3*res
                    origin_y = 0.55#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value



            if gal=='GC':
                #manually put in the size to correct for the 360->0 difference and just because it doesnt seem to work
                print("LA",((sc.longitude_extrema[0])),(sc.longitude_extrema[1]-360*u.degree),np.abs(pix_x))
                print("MA",sc.latitude_extrema,np.abs(pix_y))
                #npix_x   =int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
                #npix_y   =int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                npix_x   =int(3/np.abs(pix_x))
                npix_y   =int(1.2/np.abs(pix_y))
                if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                    npix_x   =int(2/np.abs(pix_x))
                    npix_y   =int(1.1/np.abs(pix_y))

                print(npix_x,npix_y)


        #Correct the header to the expected pixels for the new res

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y

        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y

        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y

        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        
        reheader['CTYPE1'] = "GLON-SIN"
        reheader['CTYPE2'] = "GLAT-SIN"
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']#Dont need these anymore, maybe?
            
            del reheader['LBOUND1']
            del reheader['LBOUND2']
            del reheader['LBOUND3']
            del reheader.cards['LBOUND1']
            del reheader.cards['LBOUND2']
            del reheader.cards['LBOUND3']
            
            reheader['LBOUND1']=0
            reheader['LBOUND2']=0
            reheader['LBOUND3']=0
            
            print("12312412")
        except Exception as e:
            print(e)
            
            print("Failed")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)
        
        # regrid cube to target pixel size
        
        
        
        #Cube_Name_Load_HCN = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+'_CMZ_HCN_J1_0_4.3_start'+str(FOV[0])+"x"+str(FOV[1])+'pc_'+str(3.3)+'_vel_res.fits'
        #del reheader
        #reheader = SpectralCube.read(Cube_Name_Load_HCN).hdu.header
        #reheader["CTYPE3"] = "VRAD"
        
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        del sc
        #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
        new = SpectralCube(data=sc_K_kms.hdu.data,wcs =WCS(sc_K_kms.header),header=sc_K_kms.header,mask=sc_K_kms.mask)
        new.allow_huge_operations=True
        new = new*sc_K_kms[0][0][0].unit
        #do this because scs dont like being modified
        del sc_K_kms
        sc_K_kms = new
        del new

        print(np.nanmax(sc_K_kms),np.shape(sc_K_kms))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
        

        NGC_CO_J3_2 = sc_K_kms
        del sc_K_kms
        NGC_CO_J3_2.allow_huge_operations=True

        #do this again to crop the extra pixels off

        sc=NGC_CO_J3_2

        sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc


        #Write the intermediary cubes that will then be spliced together
        del NGC_CO_J3_2
        

        

        fwhm_factor = np.sqrt(8*np.log(2))

        vel,RA,Dec =sc.world[:,0,0]
        for km in range(5,6):
            scWsc_copy = sc
            vel_prime = min_vel*((km*iter_factor+1))
            if(km==5):
                vel_prime=2.5
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            if(gal=="GC"):
                vel = np.arange(-300,301,vel_prime)*u.km/u.s    
            else:
                vel = np.arange(0,501,vel_prime)*u.km/u.s    

            G_width = np.sqrt(2.5**2-1**2)
            if(km==5):
                #G_width = 4
                G_width = np.sqrt(2.5**2-1**2)
                if(Match_to_HCO):
                    G_width = np.sqrt(3.3**2-1**2)
            if(km==5 and Smoothe_4):
                G_width = 4
            if(km==5 and Smoothe_2_5):
                G_width = 2.5/1#G_width = 2.5/1
                
            scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
            scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)
            scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range 



            scWsc_copy.write(Cube_Name_Save,overwrite=True)
            gc.collect()
            #scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32.fits",overwrite=True)
        del scWsc_copy
        
        del sc
        
        gc.collect()######################################################################
    except Exception as e:
        print(e)
        print(kl)
        print("Failed")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)


      
    
print("done")

# 6.1 Splicing

In [ ]:
#Splice right and left




#Splice CO
gal="GC"
Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_CMZ_CO_J3_2_4.3_start'

ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
iter_factor = 1/5


FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution

Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"
    
    
if(Line_Name == '_CMZ_CO_J3_2_4.3_start'):

    for kl in range(5,6):
      
        Prime_Beam = Min_res*((kl*iter_factor+1))#
        if(kl==5):
            Prime_Beam = 3*u.pc#Nico
            FOVp=[200,800]
        else:
            FOVp=FOV
            
        print(Prime_Beam)
    
        for km in range(5,6):
            vel_prime = min_vel*((km*iter_factor+1))
            if(km==5):
                vel_prime=2.5
                ovs=5
                
            
            Side="LH"
            Cube_Name_Load_LH = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            Side="RH"
            Cube_Name_Load_RH =str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            Side="FRH"
            Cube_Name_Load_FRH = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'
            Side="FLRH"
            Cube_Name_Load_FLH = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+Side+'_vel_res_NEW'+Smoothe_title+'.fits'

            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'

            SCR = SpectralCube.read(Cube_Name_Load_RH)
            SCR = SCR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
            vel,RA,Dec = SCR.world[:,0,0]
            print(vel)
            SCL=SpectralCube.read(Cube_Name_Load_LH)
            SCL = SCL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
            vel,RA,Dec = SCL.world[:,0,0]
            
            SCFR=SpectralCube.read(Cube_Name_Load_FRH)
            SCFR = SCFR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
            vel,RA,Dec = SCFR.world[:,0,0]
            
            SCFL=SpectralCube.read(Cube_Name_Load_FLH)
            SCFL = SCFL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
            vel,RA,Dec = SCFL.world[:,0,0]
            

            print(SCR[0][200][200],SCFR[0][200][200],SCL[0][200][200],SCFL[0][200][200])
           
            
            
            if gal=='GC' and Line_Name=='_CMZ_CO_J3_2_4.3_start' and kl==5:
                reheader = copy.deepcopy(SCFR.hdu.header)
                
                dist=8.178*10**3#pc
                res=  Prime_Beam/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
                ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

                if reheader['cdelt1']>0:
                    #pix_x    = sc.header['cdelt1']/5#(res/5.).to(u.degree).value*10
                    pix_x    = (res/ovs).to(u.degree).value
                    origin_x = 357
                    #pix_x    = .55/3*res.to(u.degree).value
               

                else:
                    #pix_x    = sc.header['cdelt1']/5#-1.*(res/5.).to(u.degree).value*10
                    pix_x    = -(res/ovs).to(u.degree).value
                    origin_x = 3
                    #pix_x    = -.55/3*res.to(u.degree).value

                if reheader['cdelt2']>0:
                    #pix_y    = sc.header['cdelt2']/5#(res/5.).to(u.degree).value*10
                    pix_y    = (res/ovs).to(u.degree).value
                    origin_y = -.5#sc.latitude_extrema[0].to(u.degree).value
                    #origin_y = sc.latitude_extrema[0].to(u.degree).value
                    if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                        #pix_y    = .55/3*res.to(u.degree).value
                        origin_y = -0.6#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

                else:
                    #pix_y    = sc.header['cdelt2']/5#-1.*(res/5.).to(u.degree).value*10
                    pix_y    = -(res/ovs).to(u.degree).value
                    #pix_y    = -.55/3*res.to(u.degree).value
                    origin_y = .5#sc.latitude_extrema[1].to(u.degree).value
                    #origin_y = sc.latitude_extrema[1].to(u.degree).value
                    origin_y = 0.6#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value



                if gal=='GC':
                    
                    #npix_x   =int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
                    #npix_y   =int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                    npix_x   =int(3/np.abs(pix_x))
                    npix_y   =int(1.2/np.abs(pix_y))
                    if(Line_Name == '_CMZ_CO_J3_2_4.3_start' and kl==5):
                        npix_x   =int(6/np.abs(pix_x))
                        npix_y   =int(1.2/np.abs(pix_y))

                    print(npix_x,npix_y)


                #Correct the header to the expected pixels for the new res

                reheader['cdelt1'] = pix_x
                reheader['cdelt2'] = pix_y

                reheader['naxis1'] = npix_x
                reheader['naxis2'] = npix_y

                reheader['crval1'] = origin_x
                reheader['crval2'] = origin_y

                reheader['crpix1'] = 0
                reheader['crpix2'] = 0

                reheader['CTYPE1'] = "GLON-SIN"
                reheader['CTYPE2'] = "GLAT-SIN"
                try:
                    del reheader['lonpole']
                    del reheader['latpole']
                    del reheader['wcsaxes']#Dont need these anymore, maybe?

                    del reheader['LBOUND1']
                    del reheader['LBOUND2']
                    del reheader['LBOUND3']
                    del reheader.cards['LBOUND1']
                    del reheader.cards['LBOUND2']
                    del reheader.cards['LBOUND3']

                    reheader['LBOUND1']=0
                    reheader['LBOUND2']=0
                    reheader['LBOUND3']=0

                    print("12312412")
                except Exception as e:
                    print(e)

                    print("Failed")
                    print("-"*60)
                    traceback.print_exc(file=sys.stdout)


                    # regrid cube to target pixel size



                #Cube_Name_Load_HCN = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+'_CMZ_HCN_J1_0_4.3_start'+str(FOV[0])+"x"+str(FOV[1])+'pc_'+str(3.3)+'_vel_res.fits'
                #del reheader
                #reheader = SpectralCube.read(Cube_Name_Load_HCN).hdu.header
                #reheader["CTYPE3"] = "VRAD"
                
                
                SCR.write("test1.fits",overwrite=True)
                
                SCR.write("test8.fits",overwrite=True)
                SCRN = SCR.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
                SCRN.write("test2.fits",overwrite=True)
                del SCR
                #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
                SCR = SpectralCube(data=SCRN.hdu.data,wcs =WCS(SCRN.header),header=SCRN.header,mask=SCRN.mask)
                SCR.allow_huge_operations=True
                SCR = SCR*SCRN[0][0][0].unit
                SCR.write("test3.fits",overwrite=True)
                
                del SCRN
                SCLN = SCL.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
                del SCL
                #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
                SCL = SpectralCube(data=SCLN.hdu.data,wcs =WCS(SCLN.header),header=SCLN.header,mask=SCLN.mask)
                SCL.allow_huge_operations=True
                SCL = SCL*SCLN[0][0][0].unit
                del SCLN
                SCFRN = SCFR.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
                del SCFR
                #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
                SCFR = SpectralCube(data=SCFRN.hdu.data,wcs =WCS(SCFRN.header),header=SCFRN.header,mask=SCFRN.mask)
                SCFR.allow_huge_operations=True
                SCFR = SCFR*SCFRN[0][0][0].unit
                del SCFRN
                SCFLN = SCFL.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
                del SCFL
                #new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
                SCFL = SpectralCube(data=SCFLN.hdu.data,wcs =WCS(SCFLN.header),header=SCFLN.header,mask=SCFLN.mask)
                SCFL.allow_huge_operations=True
                SCFL = SCFL*SCFLN[0][0][0].unit
                del SCFLN
            
            
            
            #Get right shape
            
            datn = SCR.hdu.data
            sx0,ex0,sy0,ey0=Crop_Nans(datn)
            
            SCR = SCR[:,sx0:ex0,:]
            
            SCR_Hdu=SCR.hdu
            zeros=((SCR_Hdu.data[:,:,:]==0))
            bp = np.where(zeros)
            SCR_Hdu.data[bp]=np.nan
            
            for lmi in range(len(datn)):
                SCR_Hdu.data[lmi,:,sy0-3:sy0+3]=np.nan
                SCR_Hdu.data[lmi,:,ey0-3:ey0+3]=np.nan
                #SCR_Hdu.data[lmi,:,ey:ey]=np.nan
            datn=SCR_Hdu.data
            sx,ex,sy,ey=Crop_Nans(datn)
            
            SCR = SpectralCube.read(SCR_Hdu)
            
            datn = SCL.hdu.data
            sx2,ex2,sy2,ey2=Crop_Nans(datn)
            
            
            
            SCL=SCL[:,sx0:ex0,:]
            
            SCL_Hdu=SCL.hdu
            zeros=((SCL_Hdu.data[:,:,:]==0))
            bp = np.where(zeros)
            SCL_Hdu.data[bp]=np.nan
            
            for lmi in range(len(datn)):
                #SCL_Hdu.data[lmi,:,sy-5:sy+5]=np.nan
                
                #SCL_Hdu.data[lmi,:,sy2-3:sy2+3]=np.nan
                SCL_Hdu.data[lmi,:,ey2-3:ey2+3]=np.nan
                
                SCL_Hdu.data[lmi,:,ey2:ey]=np.nan
                
            SCL = SpectralCube.read(SCL_Hdu)
            
            
            datn = SCFR.hdu.data
            sx3,ex3,sy3,ey3=Crop_Nans(datn)
            
            
            
            SCFR=SCFR[:,sx0:ex0,:]
            
            SCFR_Hdu=SCFR.hdu
            zeros=((SCFR_Hdu.data[:,:,:]==0))
            bp = np.where(zeros)
            SCFR_Hdu.data[bp]=np.nan
            
            
            for lmi in range(len(datn)):
                
                SCFR_Hdu.data[lmi,:,sy3-3:sy3+3]=np.nan
                SCFR_Hdu.data[lmi,:,ey3-3:ey3+3]=np.nan
                
                
                SCFR_Hdu.data[lmi,:,sy:sy3]=np.nan
                #SCFR_Hdu.data[lmi,:,ey-5:ey+5]=np.nan#Takes care of the cube edges
                
                
            SCFR = SpectralCube.read(SCFR_Hdu)
                        

            
            
            
            datn = SCFL.hdu.data
            sx4,ex4,sy4,ey4=Crop_Nans(datn)
            
            
            
            SCFL=SCFL[:,sx0:ex0,:]
            
            SCFL_Hdu=SCFL.hdu
            zeros=((SCFL_Hdu.data[:,:,:]==0))
            bp = np.where(zeros)
            SCFL_Hdu.data[bp]=np.nan
            
            
            for lmi in range(len(datn)):
                pass
                #SCFL_Hdu.data[lmi,:,sy4-3:sy4+3]=np.nan
                #SCFL_Hdu.data[lmi,:,ey4-3:ey4+3]=np.nan
                
                
                #SCFL_Hdu.data[lmi,:,sy2:sy4]=np.nan
                #SCFR_Hdu.data[lmi,:,ey-5:ey+5]=np.nan#Takes care of the cube edges
                
                
            SCFL = SpectralCube.read(SCFL_Hdu)
            #SCR.write("test4.fits",overwrite=True)
            #SCL.write("test5.fits",overwrite=True)
            #SCFR.write("test6.fits",overwrite=True)


            RD = SCR.hdu.data
            LD=SCL.hdu.data
            FRD = SCFR.hdu.data
            FLD = SCFL.hdu.data
            print(np.shape(RD),len(RD))
            print(np.shape(LD))
            print(np.shape(FRD))
            print(np.shape(FLD))
            C = np.copy(LD)
            wcs = SCR.wcs
            header=SCR.hdu.header
            hdu=SCR.hdu
            world=SCR.world


            for lmi in range(len(RD)):
                for lmj in range(len(RD[lmi])):
                    for lmk in range(len(RD[lmi][lmj])):
                        if((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0)):
                            C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],LD[lmi][lmj][lmk]])
                        elif((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (FRD[lmi][lmj][lmk]> 0 or FRD[lmi][lmj][lmk]< 0)):
                            C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],FRD[lmi][lmj][lmk]])
                        elif((LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0) and (FLD[lmi][lmj][lmk]> 0 or FLD[lmi][lmj][lmk]< 0)):
                            C[lmi][lmj][lmk] = np.nanmean([LD[lmi][lmj][lmk],FLD[lmi][lmj][lmk]])
                        elif(RD[lmi][lmj][lmk]>0 or RD[lmi][lmj][lmk]<0):
                            C[lmi][lmj][lmk] =RD[lmi][lmj][lmk]
                        elif(FRD[lmi][lmj][lmk]>0 or FRD[lmi][lmj][lmk]<0):
                            C[lmi][lmj][lmk] =FRD[lmi][lmj][lmk]
                            #print("Ye",lmi,lmj,lmk)
                        elif(LD[lmi][lmj][lmk]>0 or LD[lmi][lmj][lmk]<0):
                            C[lmi][lmj][lmk] =LD[lmi][lmj][lmk]
                        elif(FLD[lmi][lmj][lmk]>0 or FLD[lmi][lmj][lmk]<0):
                            C[lmi][lmj][lmk] =FLD[lmi][lmj][lmk]






                print(np.nanmean(C[lmi]))
                print(lmi)


            Comb = SpectralCube(data=C, wcs=wcs,header=header)
            Comb.allow_huge_operations=True
            Comb = Comb*SCL[0][0][0].unit



            Comb.write(Cube_Name_Save,overwrite=True)
print("done")

# 6.2 NAN fixing

In [21]:
#Fix the Nans

#Splice CO
gal="GC"
Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_CMZ_CO_J3_2_4.3_start'

ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
iter_factor = 1/5


FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution


Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"
    
if(Line_Name == '_CMZ_CO_J3_2_4.3_start'):

    for kl in range(5,6):
      
        Prime_Beam = Min_res*((kl*iter_factor+1))#
        if(kl==5):
            Prime_Beam = 3*u.pc#Nico
            FOVp=[200,800]
        else:
            FOVp=FOV
        print(Prime_Beam)
    
        for km in range(5,6):
            vel_prime = min_vel*((km*iter_factor+1))
            if(km==5):
                vel_prime=2.5
            
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
            Cube_Name_Load = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
            
            Cube_Name_Save=str("Cropped_"+Cube_Name_Load)
            print(Cube_Name_Load)
            sc = SpectralCube.read(Cube_Name_Load)
            sc.write("test7.fits",overwrite=True)
            datn = sc.hdu.data
            
            #Get right shape
            sx,sy,ex,ey=0,0,0,0
            for lmi in range(np.shape(datn[0,:,:])[0]):

                if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
                    print("F",lmi)
                    break
                for lmj in range(np.shape(datn[0,:,:])[1]):

                    if(sx==0):            
                        if(np.nanmean(datn[0,lmi,:])>0 or np.nanmean(datn[0,lmi,:])<0):
                            sx=lmi


                    if(sy==0):
                        if(np.nanmean(datn[0,:,lmj])>0 or np.nanmean(datn[0,:,lmj])<0):
                            sy=lmj

                    if(ex==0):
                        if(np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])<0):
                            ex=np.shape(datn[0,:,:])[0]-lmi-1

                    if(ey==0):
                        if(np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])>0 or np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])<0):
                            ey=np.shape(datn[0,:,:])[1]-lmj-1

                    if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                        break
            print(sx,ex,sy,ey)
            
            
            sc = sc[:,sx:ex,sy:ey]
            
            sc_Hdu=sc.hdu
            zeros=((sc_Hdu.data[:,:,:]==0))
            bp = np.where(zeros)
            sc_Hdu.data[bp]=np.nan
            sc = SpectralCube.read(sc_Hdu)
            sc.write("test8.fits",overwrite=True)
            
            #Crop vel axis if needed
            
            sp=0
            for lmi in range(len(sc)):
                #Check to see if the slice has been repeated by the interpolation function
                if(np.round(np.nanmean(sc[lmi].hdu.data),5)==np.round(np.nanmean(sc[lmi+1].hdu.data),5)):

                    sp = lmi+1
                else:
                    print("A")
                    print(lmi,np.nanmean(sc[lmi].hdu.data),np.nanmean(sc[lmi+1].hdu.data))
                    break
            l = len(sc)-1
            ep=l
            for lmi in range(l):

                if(np.round(np.nanmean(sc[l-lmi].hdu.data),5)==np.round(np.nanmean(sc[l-lmi-1].hdu.data),5)):
                    ep = l-lmi-1

                else:
                    print("B",lmi)
                    break

            print(sp,ep)#These are the start and stop slices where the actual unique data resides

            sc.allow_huge_operations=True
            sc = sc[sp:ep]

            
            sc.write(Cube_Name_Save,overwrite=True)
            del sc
            del sc_Hdu
            gc.collect()
            print(kl,km,'ready')
print("done")

3.0 pc
1 1174 1 1362
5 5 ready
done


# 6.3 Noise matching

In [8]:
#match noise



iterations = 6
iter_factor = 1/5

ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution
Min_beam_req = 1/5

Params = [iterations,iter_factor,ovs,min_vel,FOV,Min_res,Min_beam_req]
savePath='/home/ben/Documents/Grad Stuff/MM data/Result Files'




Line_Name = '_CMZ_CO_J3_2_4.3_start'
Line_Name_NGC = '_NGC_CO_J3_2_'



Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"

for kl in range(5,6):
      
    Prime_Beam = Min_res*((kl*iter_factor+1))#
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        FOVp=[200,800]
            
    print(Prime_Beam)

    for km in range(5,6):
        vel_prime = min_vel*((km*iter_factor+1))
        if(km==5 and kl==5):
            vel_prime=2.5
            NGCCO32_Noise=.115

        '''
        #Make subcube

        #Find noise for ngc253
        Cube_Name_Load = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name_NGC+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res.fits'
        path = Cube_Name_Load
        
        Qp = SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="radio") 
        Qp.allow_huge_operations=True

        Qp = Qp.to(u.K)#Jy to Kelvin

        datn = Qp.hdu.data
        del Qp


        Non_nan=((datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]>0)  | (datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]<0 ))
        
        if(kl==5 and km==5):
            pass
        else:
            NGCCO32_Noise = (np.nanstd(datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)],where= Non_nan)) #Noise K
        print(np.shape(datn))
        del datn
        
        '''
        
        
        
        Cube_Name_Load = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
        print(Cube_Name_Load)
        Cube_Name_Save = "Noise_Matched_"+"Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
        path = Cube_Name_Load
        
        #Qp = SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="radio") 
        Qp = SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="optical") 
        
        Qp.allow_huge_operations=True
        Q = Qp.to(u.K)#Jy to Kelvin
        del Qp
        datn=Q.hdu.data
        Non_nan=((datn[0,:,0:int(np.shape(datn)[2]/2)]>0)  | (datn[0,:,0:int(np.shape(datn)[2]/2)]<0 ))
        print(np.shape(datn))
        m = (np.nanstd(datn[0,:,0:int(np.shape(datn)[2]/2)],where= Non_nan)) #Noise K
        
        m=.037

        print(m,"Noise (K) matched to ",NGCCO32_Noise)


        


        npixels = np.product(Q.hdu.data.shape)

        target_noise = float(NGCCO32_Noise)
        actual_noise = m
        additional_sigma = np.sqrt(target_noise**2 - actual_noise**2)

        additional_noise = np.random.normal(0., additional_sigma, npixels)
        additional_noise = np.reshape(additional_noise, Q.hdu.data.shape)
        
        
        fwhm_factor = np.sqrt(8*np.log(2))
        add_noise = np.zeros(np.shape(datn))
        for lmi in range(len(datn)):
            new_seed = np.random.randint(1e9)
            additional_noise = gauss_correlated_noise_2D(shape=(Q.hdu.data[6].shape[0],Q.hdu.data[6].shape[1]), sigma=additional_sigma, beam_gauss_width=5/fwhm_factor,randomseed=new_seed)
            pp=np.where(additional_noise!=np.nan)
            add_noise[lmi][pp]=additional_noise[pp]
            print(lmi)
        
        new_data = datn+add_noise
        QCopy = Q.hdu
        QCopy.data = new_data
        Q = SpectralCube.read(QCopy)
        del QCopy

        
        Q.write(Cube_Name_Save,overwrite=True)
        del Q
        print(kl,km)
print("Doen")

(194, 982, 1279)
0.305 Noise (K) matched to  0.115


KeyboardInterrupt: 

# Part 8: Dendograms

In [4]:
#CO 3-2 all resses starting at 4.3


Num=5
Overlaps=0#4
Ram_Limiter=1#What percent of the cube my ram can handle
LineN="CMZ_CO_J3_2_all_NEW"
Name = "CMZ_CO_J3_2_all_NEW"
name = "CMZ_CO_J3_2_CM_Contours_all_NEW.jpeg"

Abs_Levels = ["All","None","No Clusters", "None (m)", "None All Channels"]



iterations = 6
iter_factor = 1/5
#Line_Name = '_CMZ_CO_J3_2_4.3_start'

NM= True

Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"

Edge_cases=True

if(Edge_cases):
    Edge_title="Edges"
else:
    Edge_title=''
    
Line_Name = '_CMZ_CO_J3_2_4.3_start'

if(NM):
    Line_Name_Save = Line_Name+'_NM'+Smoothe_title+Edge_title
else:
    Line_Name_Save=Line_Name+Smoothe_title+Edge_title



ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution
Min_beam_req = 1/5

Params = [iterations,iter_factor,ovs,min_vel,FOV,Min_res,Min_beam_req]
savePath='/home/ben/Documents/Grad Stuff/MM data/Result Files'
if(NM):
    #Name_File = "CMZ_Names_New_All_Kinds_NM_Fixed"
    Name_File = "CMZ_Names_New_All_Kinds_NM"+Smoothe_title+Edge_title
else:
    #Name_File= "CMZ_Names_New_All_Kind_Fixed"
    Name_File= "CMZ_Names_New_All_Kinds"+Smoothe_title+Edge_title

np.savetxt(Line_Name+"_Params", Params,fmt='%s')#[iterations,iter_factor ,Line_Name,Name,ovs,min_vel,FOV,Min_res,Min_beam_req]


cen_p1 = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
cen_p2 = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')

In [22]:
savePath='/home/ben/Documents/Grad Stuff/MM data/Result Files'

savePath="/Users/b347m182/Documents/Grad Stuff/MM Data/Result Files"

In [21]:
#Krieger

#Beam
Abs_Level = Abs_Levels[0]



if gal =="GC":
    pathCont = '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits'#'CMZ_Continuum_Splice.fits'
else:
    pathCont = 'Continuum_Reproject.fits'





start_num=0
km=0
for kl in range(5,6):
      
    Prime_Beam = Min_res*((kl*iter_factor+1))#
    max_size = Prime_Beam*7   
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        km=5
        FOVp=[200,800]
        max_size = 20#pc
    else:
        FOVp=FOV
    print(Prime_Beam)
    
    
    vel_prime = min_vel*((km*iter_factor+1))

    if(km==5):
        vel_prime=2.5
    if(kl==5,km==5):
        Trunks=True
    else:
        Trunks=True




    try:
        Names=list(np.load(Name_File+".npy",allow_pickle=True))
        if(iterations!=np.shape(Names)[3]):
            del popasasd
    except Exception as e:

        print(e)
        print("initialized names")
        Names = list(np.empty((20,iterations,iterations,iterations,iterations,iterations,iterations),dtype=object))


    #Continuum image
    scCont = spectral_cube.Projection.from_hdu(fits.open(pathCont)[0])
    #Put this up here for the column density map
    metadata = {}
    metadata["distance"] = dist_cmz
    arc_per_pix_yc =  abs(scCont.hdu.header["CDELT1"]*3600.0 * u.arcsec)/u.pix
    arc_per_pix_xc =  abs(scCont.hdu.header["CDELT2"]*3600.0 * u.arcsec)/u.pix
    beam_majorc =  scCont.hdu.header["BMAJ"]*3600.0 * u.arcsec
    beam_minorc =  scCont.hdu.header["BMIN"]*3600.0 * u.arcsec
    beam_area_ratioc = beam_minorc*beam_majorc/arc_per_pix_yc/arc_per_pix_xc*1.13309#beam_area_ratioc = beam_minorc*beam_majorc/arc_per_pix_yc/arc_per_pix_xc#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    metadata['beam_area_ratioc']=beam_area_ratioc
    pc_per_pixelc = abs(scCont.hdu.header["CDELT1"])/180*np.pi*metadata['distance']/u.Mpc
    print(pc_per_pixelc,"MPC per pix")

    #Make subcube


    if(NM):
        Cube_Name_Load = 'Noise_Matched_'+"Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
    else:
        Cube_Name_Load = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
    path = Cube_Name_Load

    Qp = SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="radio") 
    Qp.allow_huge_operations=True

    Q = Qp.to(u.K)#Jy to Kelvin

    sc = Q.unmasked_copy()



    scW = sc.wcs[:][:][0]
    dat = sc.hdu.data[int(len(sc)*((1-Ram_Limiter)/2)):int(len(sc)*(Ram_Limiter + (1-Ram_Limiter)/2)),:,:]
    scn = sc[int(len(sc)*((1-Ram_Limiter)/2)):int(len(sc)*(Ram_Limiter + (1-Ram_Limiter)/2)),:,:]
    scF = scn
    datn = dat


    #m=.115

    #print(m,"Presumed Noise (K)")

    #Continuum image
    scCont = spectral_cube.Projection.from_hdu(fits.open(pathCont)[0])
    #Do the same thing to the continuum image
    scCont.allow_huge_operations=True
    scContW = scCont.reproject(scF.moment0().header)
    #m=.115


    scW = sc.wcs[:][:][0]




    #Do the same thing to the continuum image

    Continuum_Data  =scContW.hdu.data
    scWCS = scF.wcs[:][:][0]

    moment_0_sub  = scF.to(u.K).moment(order=0,how='slice')            # Calculate the Moment 0 map 



    ######Moment 0 for both
    ######and cont
    Make_Plot((LineN+" Moment 0"),"Moment 0 (K km/s)",moment_0_sub.hdu.data,0,np.nanmax(moment_0_sub.hdu.data),moment_0_sub.wcs,1,1,1,True)


    cSD = (Flux_to_Mass(scContW.hdu.data*u.Jy/u.beam/beam_area_ratioc*u.pix**2*u.beam)/(pc_per_pixelc*10**6*3.086*10**18*u.cm)**2)*(1.989*10**30*u.kg/u.M_sun)*Num_per_kg/u.kg #Flux goes to luminosity for a gaussian beam and goes to column density for square pixels, mass goes to kg to number





    rm=moment_0_sub.hdu.data/cSD
    rmU = rm*u.K*u.km/u.s#Just put back in the units
    rmU = np.array(rmU /(u.K*u.km/u.s)/u.cm**2*10**22,dtype='float64')#Now remove them all
    print(rmU)

    ######ratio

    bp = np.where(cSD<=.0001*10**22/u.cm**2)
    bp2 = np.where( moment_0_sub.hdu.data < .00022)
    Continuum_Data[bp] = np.nan
    cSD[bp]=np.nan
    rmU[bp]=np.nan
    rmU[bp2]=np.nan

    Non_nan=((datn[0,:,0:int(np.shape(datn)[2]/2)]>0)  | (datn[0,:,0:int(np.shape(datn)[2]/2)]<0 ))

    m = (np.nanstd(datn[0,:,0:int(np.shape(datn)[2]/2)],where= Non_nan)) #Noise K
    mp=m
    if(Abs_Level=="All"):
        pass
    if(Abs_Level=="None"):
        datn[np.where(datn<0)]=np.nan
    if(Abs_Level=="None (m)"):
        datn[np.where(datn<-m)]=np.nan
    if(Abs_Level=="None All Channels"):
        for lmi in range(len(datn)):
            bpP = np.where(datn[lmi]<-m)
            for lmj in range(len(datn)):
                datn[lmj][bpP]=np.nan
    if(Abs_Level=="No Clusters"):

        #IDs,RAs,Decs,HWHM  =    np.genfromtxt("NGC_Clusters_"+"IDs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"RAs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"Decs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"HWHM"+"")

        HWHM_rad,CDs_New,glons_New,glats_New,IDs_New   =    np.genfromtxt("CMZ_Clusters_"+"HWHM"+""),np.genfromtxt("CMZ_Clusters_"+"CD"+""),np.genfromtxt("CMZ_Clusters_"+"Glon"+""),np.genfromtxt("CMZ_Clusters_"+"Glat"+""),np.genfromtxt("CMZ_Clusters_"+"IDs"+"")

        datn=  Mask_Clusters_CMZ(HWHM_rad,scWCS,scF.hdu.header,datn,glons_New,glats_New,One_Pc=False,One_Pc_Size=1,HWHM_Fac=3)

    print(m,"Unmatched Noise (K)", "Abs = ",Abs_Level)


    Make_Plot("Column Density","(#/cm^2)",cSD,float(np.nanmin(cSD*u.cm**2)),float(np.nanmax(cSD*u.cm**2)),scCont.wcs,1,1,1,True)
    Make_Plot((LineN+" Moment 0/Column Density"),"Moment 0 over Column Density of the Continuum (K km/s/(#/cm^2))",rmU,np.nanmean(rmU)*.5,abs(np.nanmean(rmU))*8,scCont.wcs,1,1,1,True)

    print(np.nanmin(rmU))



    header = scF.hdu.header
    print()
    #make metadata for the dendrogram

    try:
        freq = header["RESTFREQ"]*u.Hz#
        metadata['wavelength']=299792458*u.m/header["RESTFREQ"]#
        print(1,freq,metadata['wavelength'])
    except:
        freq = header["RESTFRQ"]*u.Hz#
        metadata['wavelength']=299792458*u.m/header["RESTFRQ"]#
    metadata['data_unit'] =scF[0][0][0].unit# header['BUNIT']
    metadata['arc_per_pix_y'] =  abs(header["CDELT1"]*3600.0 * u.arcsec)/u.pix
    metadata['arc_per_pix_x'] =  abs(header["CDELT2"]*3600.0 * u.arcsec)/u.pix


    metadata['beam_major'] =  header["BMAJ"]*3600.0 * u.arcsec
    metadata['beam_minor'] =  header["BMIN"]*3600.0 * u.arcsec
    beam_area_ratio = metadata['beam_minor']*metadata['beam_major']/metadata['arc_per_pix_y']/metadata['arc_per_pix_x']*1.13309#beam_area_ratio = metadata['beam_minor']*metadata['beam_major']/metadata['arc_per_pix_y']/metadata['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    metadata['beam_area_ratio']=beam_area_ratio
    metadata['spatial_scale'] = np.sqrt(abs(header["CDELT1"])*u.degree**2*abs(header["CDELT2"]))
    #area_res = abs(header["CDELT1"])*abs(header["CDELT2"])*(np.pi/180*3.5)**2#mpc^2/pix^2
    #print(area_res,type(area_res))

    print(metadata['beam_minor'],metadata['beam_major'])
    print(beam_area_ratio)
    #metadata["wcs"] = wcs
    metadata["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#u.km/u.s

    metadata["vaxis"]=0
    metadata["wcsu"]=scF.wcs


    #for k3 in range(start_num):
    k3=2


    beam_req = Min_beam_req*(k3+1)
    if kl==5:
        beam_req=.9/3
    k4=0
    print(kl,km,k3,k4)
    pix_thresh_factor = k4+1
    k6=2
    delt_factor = (3/5+k6/5)

    k7=2
    noise_factor = (3/5+k7/5)
    if(kl==5):
        m=.115
    else:
        m=mp
    print(kl,km,k3,k4,k6,k7)
    print("actiev noise = ",m)
    try:



        Cube_Name_Save = Name+"_Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name_Save+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_'+str(beam_req)+"_beam_size_req_"+str(pix_thresh_factor)+'_times_beam_pixels_'+str(delt_factor)+"_delt_factor_"+str(noise_factor*5)+"_SNR_NEW.fits"



        Names[Num][kl][km][k3][k4][k6][k7] = Cube_Name_Save

        NameS = (Cube_Name_Save+"Sigmas")
        NameR = (Cube_Name_Save+"Radii")
        NameCol = (Cube_Name_Save+"_Column")
        NameLum = (Cube_Name_Save+"_Luminosities")

        NameFlux = (Cube_Name_Save+"Flux_Dense")
        NameRFF = (Cube_Name_Save+"Rad_For_Flux")

        print(Cube_Name_Save)

        #np.save("CMZ_Names_New_All_Kinds",Names)

        np.save(Name_File,Names)




        d = astrodendro.Dendrogram.compute(datn,min_delta=m*delt_factor,min_value=m*5*(noise_factor),min_npix=beam_area_ratio.value*pix_thresh_factor) #The main culprit
        #d = astrodendro.Dendrogram.compute(datn,min_delta=m,min_value=m*5,min_npix=beam_area_ratio.value) #The main culprit

        vel,RA,Dec = scF.world[:,0,0]
        Rads,Sigmas,CD,Lumin,SIDS,MOM0FLUX,Distances,V_err = Dendro_Arrays(d,datn,vel,Continuum_Data,metadata,beam_size=Prime_Beam.value,beam_req=beam_req,Trunks=Trunks,max_size =max_size,edge_cases=Edge_cases )


        ##Analyze dendograms
        pc_per_pixel = abs(header["CDELT1"])/180*np.pi*metadata['distance']/u.Mpc

        sig_per_pixel=np.nan#metadata['spectral_resolution']/u.pix


        #Finds Size, Linewidth, Luminosity, and Column Density of a structure for each structure and returns them in [[][]] arrays [Leaves][Branches]


        from scipy.optimize import curve_fit
        from scipy.optimize import leastsq

        def func(R,a,b):
            return a*R**(b)
        try:
            popt, pcov = curve_fit(func, Rads[0]*10**6,Sigmas[0])
        except:
            popt, pcov = [np.nan,np.nan],[[np.nan,np.nan],[np.nan,np.nan]]
        try:
            poptB, pcovB = curve_fit(func,  Rads[1]*10**6,Sigmas[1])
        except:
            poptB, pcovB = [np.nan,np.nan],[[np.nan,np.nan],[np.nan,np.nan]]



        Rcon = np.concatenate((Rads[0],Rads[1]))


        Scon = np.concatenate((Sigmas[0],Sigmas[1]))
        Scon2 = np.concatenate((Sigmas[2],Sigmas[3]))
        print(np.shape(Rcon))

        poptCon, pcovCon = curve_fit(func, Rcon*10**6, Scon)










        xs=np.linspace(np.nanmin(Rcon*10**6),np.nanmax(Rcon*10**6),50)

        ysL,ysB,ysCon=func(xs,popt[0],popt[1]),func(xs,poptB[0],poptB[1]),func(xs,poptCon[0],poptCon[1])


        LuminCon = np.concatenate((Lumin[0],Lumin[1]))
        ColumnCon= np.concatenate((CD[0],CD[1]))

        MOM0FLUXcon = np.concatenate((MOM0FLUX[0],MOM0FLUX[1]))
        DistancesCon= np.concatenate((Distances[0],Distances[1]))
        print(Distances)

        #Radius Luminosity fit
        #Good pixels are places where the luminosity and therefore column density are above zero
        gp = np.where(LuminCon>0)
        print(np.shape(Rcon),np.shape(LuminCon))
        lgp = LuminCon[gp]
        radgp = Rcon[gp]*10**6

        gp2 = np.where(ColumnCon>0)
        cgp = ColumnCon[gp2]
        rgp=Scon[gp2]**2/Rcon[gp2]/10**6
        dgp=DistancesCon[gp2]
        radgpFORRAT = Rcon[gp2]*10**6
        siggpFORRAT = Scon[gp2]
        lumFORRAT = LuminCon[gp2]
        mom0FORRAT = MOM0FLUXcon[gp2]

        #print(np.shape(Rcon[gp]*10**6),np.shape(LuminCon[gp]))

        RLpopt,RLpcov = curve_fit(func, np.array(radgp,dtype='float64'),np.array(lgp,dtype='float64'),maxfev=999999,p0 = np.array([10**27,1],dtype='float64'),ftol = 10**-10)

        xsRL=np.linspace(np.nanmin(radgp),np.nanmax(radgp),50)
        ysRL = func(xsRL,RLpopt[0],RLpopt[1])
        #Column density to Size-linewidth
        CDpopt,CDpcov = curve_fit(func, np.array(cgp,dtype='float64') ,np.array(rgp,dtype='float64'),maxfev=9999099,p0 = np.array([10,.01],dtype='float64'),ftol = 10**-10)

        Dist_popt,Dist_pcov = curve_fit(func, np.array(DistancesCon,dtype='float64') ,np.array(Scon,dtype='float64'),maxfev=9999099,ftol = 10**-10)
        Dist_popt2,Dist_pcov2 = curve_fit(func, np.array(dgp,dtype='float64') ,np.array(rgp,dtype='float64'),maxfev=9999099,ftol = 10**-10)

        xsDist=np.linspace(np.nanmin(DistancesCon),np.nanmax(DistancesCon),50)
        xsDist2=np.linspace(np.nanmin(dgp),np.nanmax(dgp),50)
        ysDist=func(xsDist,Dist_popt[0],Dist_popt[1])
        ysDist2=func(xsDist2,Dist_popt2[0],Dist_popt2[1])

        gp3 = np.where(lumFORRAT>0)
        lumFORRAT = lumFORRAT[gp3]
        rgpFORRAT = rgp[gp3]
        mom0FORRAT = mom0FORRAT[gp3]

        xsCD=np.linspace(np.nanmin(cgp),np.nanmax(cgp),50)
        ysCD = func(xsCD,CDpopt[0],CDpopt[1])


        print(poptCon,pcovCon,"Size Linewidth All_structures:")
        print("a = ",poptCon[0],"+-",np.sqrt(pcovCon[0][0]))
        print("b = ",poptCon[1],"+-",np.sqrt(pcovCon[1][1]))


        print(RLpopt,RLpcov,"Radius to Luminosity:")
        print("c = ",RLpopt[0],"+-",np.sqrt(RLpcov[0][0]))
        print("d = ",RLpopt[1],"+-",np.sqrt(RLpcov[1][1]))



        print(CDpopt,CDpcov,"Column Density to linewidth^2/size:")
        print("e = ",CDpopt[0],"+-",np.sqrt(CDpcov[0][0]))
        print("f = ",CDpopt[1],"+-",np.sqrt(CDpcov[1][1]))



        gp4=np.where(MOM0FLUXcon>0)
        mom0FORFlux=MOM0FLUXcon[gp4]
        radgpFORFlux=Rcon[gp4]*10**6
        rgpFORFlux=Scon[gp4]**2/Rcon[gp4]/10**6


        '''

        '''
        fig = plt.figure(1,figsize=(40,40))


        axAlpha = pylab.subplot(5, 5, 7)
        axBeta = pylab.subplot(5, 5, 8)
        axGamma = pylab.subplot(5, 5, 9)
        axDelta = pylab.subplot(5, 5, 10)




        p1=d.plotter()
        p1.plot_tree(axAlpha)
        axAlpha.set_xlabel("Structure")
        axAlpha.set_ylabel("Flux (K)")
        axAlpha.set_title("Whole data set")



        #Plot contours for the top ten clusters


        #scCropped =scF.moment0().hdu.data
        scCropped =scF.moment0().hdu.data
        scCropped[np.where(scF.moment0().hdu.data<.22)]=np.nan
        print(np.nanmean(np.nanstd(scF.moment0().hdu.data))*1)
        scCropped[bp]=np.nan
        axDelta = pylab.subplot(5,5,8,projection=scF.moment0().wcs)
        #axBeta.imshow(scCropped.moment0().hdu.data, origin='lower', interpolation='nearest', cmap=plt.cm.Blues,norm=colors.LogNorm(vmin=5))
        imBeta = axDelta.imshow(scCropped, origin='lower', interpolation='nearest', cmap=plt.cm.Blues,vmin=0,vmax = np.nanmax(scCropped)*.7 )



        ColumnCon= np.concatenate((CD[0],CD[1]))
        SIDScon = np.concatenate((SIDS[0],SIDS[1]))
        nth =0# sorted(CD[0])[len(CD[0])-20]

        G1 = True

        RA = axDelta.coords[0]                                                                  # 
        Dec = axDelta.coords[1]

        RA.set_ticks(size=-3)                                                                                      
        Dec.set_ticks(size=-3) 
        RA.set_ticklabel(exclude_overlapping=True) 
        Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
        axDelta.tick_params(axis = 'both', which = 'major', labelsize = 15)    
        cb=pylab.colorbar(imBeta,fraction=0.016,pad=0.04)                                     
        cb.set_label(label="Moment 0 K km/s",fontsize=10,rotation=270,labelpad=20) 
        cb.ax.tick_params(which = 'major', labelsize = 10)   
        pylab.annotate(s=LineN,fontsize=10,xy=(.01,1.05),xycoords="axes fraction",c="black")  
        pylab.annotate(s="Highest Density",fontsize=10,xy=(.31,1.05),xycoords="axes fraction",c="red")  











        ax2 = pylab.subplot(5, 5, 1)
        ax3 = pylab.subplot(5, 5, 2)
        ax4 = pylab.subplot(5, 5, 3)

        xpcon = ax2.scatter(Rcon*10**6,Scon,label="All_structures",s=30,alpha=.7)


        p = ax2.plot(xs,ysCon)





        st="WDS: a="+ str(np.format_float_scientific(poptCon[0],1))+"+-"+str(np.format_float_scientific(np.sqrt(pcovCon[0][0]),1))+" b="+str(np.format_float_scientific(poptCon[1],1))+"+-"+str(np.format_float_scientific(np.sqrt(pcovCon[1][1]),2))
        ax2.annotate(s=st,fontsize=10,xy=(0.01,0.01),xycoords="axes fraction")

        ax2.annotate(s=r'$\sigma$ = $a*R^b$',fontsize=10,xy=(0.01,0.10),xycoords="axes fraction")
        ax2.set_yscale('log')
        ax2.set_xscale('log')
        ax2.set_ylim(np.nanmin(Scon),np.nanmax(Scon))
        ax2.set_xlabel("Size (pc)",fontsize=12)
        ax2.set_ylabel("Sigma (km/s)",fontsize=12)
        ax2.legend(prop={'size': 12},loc="upper right")
        ax2.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")

        #gp = np.where(FRs>0)







        xspL = ax4.scatter(radgpFORRAT,siggpFORRAT,label="All_structures",s=30)




        ax4.set_yscale('log')
        ax4.set_xscale('log')

        ax4.set_xlabel("Radius (pc)",fontsize=12)
        ax4.set_ylabel("sig (km/s)",fontsize=12)
        ax4.legend(prop={'size': 12},loc="upper right")






        xspF = ax3.scatter(radgpFORRAT,rgp,label="All_structures",s=30)

        ax3.annotate(s=st,fontsize=12,xy=(0.01,0.04),xycoords="axes fraction")
        ax3.set_yscale('log')
        ax3.set_xscale('log')
        ax3.set_ylim(np.nanmin(rgp),np.nanmax(rgp))
        ax3.set_xlabel("Rad (pc)",fontsize=12)
        ax3.set_ylabel("Linewidth Ratio (km^2/s^2/pc)",fontsize=12)
        ax3.legend(prop={'size': 12})





        ax5 = pylab.subplot(5, 5, 4)
        ax6 = pylab.subplot(5, 5, 5)
        ax7 = pylab.subplot(5, 5, 6)



        xpconhh = ax5.scatter(radgpFORRAT,cgp,label="All_structures",s=30,alpha=.7)
        ax5.set_yscale('log')
        ax5.set_xscale('log')
        ax5.set_xlabel("Size (pc)",fontsize=12)
        ax5.set_ylabel("Column Density (#/cm^2 > 7e22)",fontsize=12)
        ax5.legend(prop={'size': 12},loc="upper right")




        xspF2 = ax6.scatter(cgp,rgp,label="All_structures",s=30,alpha=.7)
        pF2 = ax6.plot(xsCD,ysCD)

        ax6.set_yscale('log')
        ax6.set_xscale('log')
        ax6.set_ylim(np.nanmin(rgp),np.nanmax(rgp))
        ax6.set_xlabel("Column Density (#/cm^2 > 7e22)",fontsize=12)
        ax6.set_ylabel("Linewidth Ratio (km^2/s^2/pc)",fontsize=12)
        ax6.legend(prop={'size': 12},loc="upper right")
        ax6.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")




        xspLu2 = ax7.scatter(radgp,lgp,label="All_structure Whole Data Set",s=30,alpha=.7)
        pFLLu2 = ax7.plot(xsRL,ysRL)

        st = "WDS, c = "+ str(np.format_float_scientific(RLpopt[0],3))+"+-"+str(np.format_float_scientific(np.sqrt(RLpcov[0][0]),3))+" d = "+str(np.format_float_scientific(RLpopt[1],3))+"+-"+str(np.format_float_scientific(np.sqrt(RLpcov[1][1]),3))
        ax7.annotate(s='L = $c*R^d$',fontsize=12,xy=(0.01,0.13),xycoords="axes fraction")
        ax7.annotate(s=st,fontsize=12,xy=(0.01,0.04),xycoords="axes fraction")

        ax7.set_yscale('log')
        ax7.set_xscale('log')
        ax7.set_ylim(np.nanmin(lgp),np.nanmax(lgp))
        ax7.set_xlabel("Radius (pc)",fontsize=9)
        ax7.set_ylabel("Luminosity (erg)",fontsize=9)
        ax7.legend(prop={'size': 12},loc="upper right")
        ax7.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")


        axLam = pylab.subplot(5, 5, 11)
        lum_rat = axLam.scatter(lumFORRAT/radgpFORRAT**2,rgpFORRAT,label="All_structure  ",s=30,alpha=.7)
        #Off by factors for area, simply using r^2

        axLam.set_yscale('log')
        axLam.set_xscale('log')
        axLam.set_xlabel("Lum/R^2 (erg/pc^2)",fontsize=9)
        axLam.set_ylabel("Linewidth Ratio (km/s)^2/pc",fontsize=9)
        axLam.legend(prop={'size': 12},loc="upper right")
        axLam.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")



        axDelta = pylab.subplot(5, 5, 10)






        mom0_rat = axDelta.scatter(mom0FORFlux/radgpFORFlux**2,rgpFORFlux,label="All_structure  ",s=30,alpha=.7)
        #Off by factors for area, simply using r^2

        axDelta.set_yscale('log')
        axDelta.set_xscale('log')
        axDelta.set_xlabel("Mom0 Flux/pc^2 (K km/s / R^2)",fontsize=9)
        axDelta.set_ylabel("Linewidth Ratio (km/s)^2/pc",fontsize=9)
        axDelta.legend(prop={'size': 12},loc="upper right")
        axDelta.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")



        axdis = pylab.subplot(5, 5, 12)
        axdis2 = pylab.subplot(5, 5, 13)
        axdis.scatter(DistancesCon,Scon)
        axdis2.scatter(dgp,rgp)
        axdis.plot(xsDist,ysDist)
        axdis2.plot(xsDist2,ysDist2)
        axdis2.set_yscale('log')
        axdis2.set_xscale('log')
        axdis.set_yscale('log')
        axdis.set_xscale('log')

        pylab.show()









        v1 = d.viewer()
        v1.show()


        #Must use text because np load is broken
        if(Abs_Level=="All"):
            Suffix=''
        if(Abs_Level=="None"):
            Suffix='_NA'
        if(Abs_Level=="No Clusters"):
            Suffix='_No_Clusters'
        if(Abs_Level=="None (m)"):
            Suffix='None_m'
        if(Abs_Level=="None All Channels"):
            Suffix='None_All_Channels'






        np.savetxt(os.path.join(savePath,NameCol+"_Leaves"+Suffix), CD[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameCol+"_Branches"+Suffix), CD[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameLum+"_Leaves"+Suffix), Lumin[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameLum+"_Branches"+Suffix), Lumin[1],fmt='%s')





        np.savetxt(os.path.join(savePath,NameS+"_Leaves"+Suffix), Sigmas[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameS+"_Branches"+Suffix), Sigmas[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameR+"_Leaves"+Suffix), Rads[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameR+"_Branches"+Suffix), Rads[1],fmt='%s')


        np.savetxt(os.path.join(savePath,NameFlux+"_Leaves"+Suffix), MOM0FLUX[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameFlux+"_Branches"+Suffix), MOM0FLUX[1],fmt='%s')



        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Leaves"+Suffix), Distances[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Branches"+Suffix), Distances[1],fmt='%s')






        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Leaves"+Suffix), V_err[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Branches"+Suffix), V_err[1],fmt='%s')

        SIDS=np.concatenate((SIDS[0],SIDS[1]))

        print(SIDS[len(SIDS)-3:len(SIDS)-1])

        
        
        import datetime
        now = str(datetime.datetime.now())
        string=now+ " "+ "Nico's method, common Gauss Smoothing (No Trunks,  min size=.9pc, delt = 1*m, m=.115K, SNR=5, max_size=21pc, G_width = sqrt(2.5^2-1),NM=true )"
        ax2.annotate(s=string,fontsize=18,xy=(0.01,1.0),xycoords="axes fraction")
    except Exception as e:
        print(e)
        print(kl,km,k3,k4)
        print("Failed")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)

        if(Abs_Level=="All"):
            Suffix=''
        if(Abs_Level=="None"):
            Suffix='_NA'
        if(Abs_Level=="No Clusters"):
            Suffix='_No_Clusters'
        if(Abs_Level=="None (m)"):
            Suffix='None_m'
        if(Abs_Level=="None All Channels"):
            Suffix='None_All_Channels'
        nans = [[np.nan,np.nan],[np.nan,np.nan]]



        np.savetxt(os.path.join(savePath,NameCol+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameCol+"_Branches"+Suffix), nans[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameLum+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameLum+"_Branches"+Suffix), nans[1],fmt='%s')





        np.savetxt(os.path.join(savePath,NameS+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameS+"_Branches"+Suffix), nans[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameR+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameR+"_Branches"+Suffix), nans[1],fmt='%s')


        np.savetxt(os.path.join(savePath,NameFlux+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameFlux+"_Branches"+Suffix), nans[1],fmt='%s')



        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Branches"+Suffix), nans[1],fmt='%s')






        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Branches"+Suffix), nans[1],fmt='%s')

3.0 pc
[Errno 2] No such file or directory: 'NGC_Names_New_All_Kinds_NM.npy'
initialized names


FileNotFoundError: [Errno 2] No such file or directory: 'Noise_Matched_Cropped_3.0pc_beam__NGC253_CO_J3_2_200x800pc_2.5_vel_res_NEW.fits'

In [ ]:

#All, all abs levels
#Newest method
#SL cmz


CMZLs = ["HCO+_1_0","HCN_1_0",'H13CN_1_0','H13CO+_1_0',"CO_3_2",'CO_J3']




Line_Names = ['_NGC_HCOp_J1_0_','_NGC_HCN_J1_0_','','','','_NGC_CO_J3_2_']


NM= True

Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"

Edge_cases=True

if(Edge_cases):
    Edge_title="Edges"
else:
    Edge_title=''




if(NM):
    #Name_File = "CMZ_Names_New_All_Kinds_NM_Fixed"
    Name_File = "CMZ_Names_New_All_Kinds_NM"+Smoothe_title+Edge_title
    NM_mod="NM"
else:
    #Name_File= "CMZ_Names_New_All_Kind_Fixed"
    Name_File= "CMZ_Names_New_All_Kinds"+Smoothe_title+Edge_title
    NM_mod=""

CMZNames=list(np.load(Name_File+".npy",allow_pickle=True))    


#CMZNames=list(np.load("CMZ_Names_New_All_Kinds.npy"))



Abs_Levels = ["All","None","No Clusters", "None (m)", "None All Channels"]
Suffixes=['','_NA','_No_Clusters','None_m','None_All_Channels']
savePath='/home/ben/Documents/Grad Stuff/MM data/Result Files'


Plot=True
for i in range(0,6):
    
    
    
    
    Params=np.genfromtxt(os.path.join(savePath,"_NGC_CO_J3_2_"+"_Params"))#[iterations,iter_factor ,Line_Name,Name,ovs,min_vel,FOV,Min_res,Min_beam_req]
    iterations,iter_factor,ovs,min_vel,FOV,Min_res,Min_beam_req = int(Params[0]),Params[1],int(Params[2]),Params[3],np.array(Params[4]),Params[5],Params[6]
    iterations=6
    for kl in range(5,6):
        
        if(Line_Names[i]==''):
            break
    
        
        
        Prime_Beam = Min_res*((kl*iter_factor+1))#
        if(kl==5):
            if(i==5):
                Prime_Beam=3
            else:
                Prime_Beam=4.3
        print(Prime_Beam)
    
    
        for km in range(5,6):
            
            vel_prime = min_vel*((km*iter_factor+1))
            
            if(km==5):
                if(i==5):
                    vel_prime=2.5
                else:
                    vel_prime=3.3
            fig = plt.figure(1,figsize=(8,8))
            gs0 = gridspec.GridSpec(len(Suffixes), 1, figure=fig)
            
            
            
            min_space=np.array([[np.full(100,Prime_Beam/ovs)],[np.linspace(-100,1000,100)]])
            Beams_s=np.array([[np.full(100,Prime_Beam)],[np.linspace(-100,1000,100)]])
            spec=np.array([[np.linspace(-100,1000,100)],[np.full(100,vel_prime)]])
            zeroes=np.array([[np.full(100,0)]])
            
            
            #for k5 in range(len(Suffixes)):
            for k5 in range(0,1):

                
                
                
                Suffix=Suffixes[k5]
                Abs_Level =Abs_Levels[k5]

                
                

                
                
                
                
                #fig.suptitle((CMZLs[i]+" Size-Linewidth Relation ("+str(Abs_Level)+")"), fontsize=18)
                #gs0.set_title((CMZLs[i]+" Size-Linewidth Relation ("+str(Abs_Level)+")"), fontsize=18)
                
                gs00 = gridspec.GridSpecFromSubplotSpec(iterations, iterations, subplot_spec=gs0[k5])

                
                

                for k3 in range(2,3):

                    beam_req = Min_beam_req*(k3+1)
                    if kl==5:
                        beam_req=.9/3
                    
                    beam_req_line = Beams_s*beam_req
                    for k4 in range(0,1):
                        for k6 in range(2,3):
                            delt_factor = (3/5+k6/5)

                            for k7 in range(2,3):
                                noise_factor = (3/5+k7/5)
                                
                                
                                print(kl,km,k3,k4,k6,k7)
                                
                                #ax = fig.add_subplot(gs00[k6, k7])
                                ax = fig.add_subplot(1,1,1)

                                try:

                                    pix_thresh_factor = k4+1

                                    Name = CMZNames[i][kl][km][k3][k4][k6][k7]
                                    print(CMZNames[i][kl][km][k3][k4])
                                    #print(Name)








                                    rname=Name+"Radii"

                                    sname=Name+"Sigmas"

                                    sEname=Name+"V_err"

                                    radsBase=[[],[]]
                                    sigsBase=[[],[]]

                                    sigs_err_Base=[[],[]]
                                    radsBase[0],radsBase[1] = np.genfromtxt(os.path.join(savePath,rname+"_Leaves"+Suffix))*10**6, np.genfromtxt(os.path.join(savePath,rname+"_Branches"+Suffix))*10**6#km/s
                                    sigsBase[0] ,sigsBase[1] = np.genfromtxt(os.path.join(savePath,sname+"_Leaves"+Suffix)),np.genfromtxt(os.path.join(savePath,sname+"_Branches"+Suffix))#km/s
                                    
                                    sigs_err_Base[0] ,sigs_err_Base[1] = np.genfromtxt(os.path.join(savePath,sEname+"_Leaves"+Suffix)),np.genfromtxt(os.path.join(savePath,sEname+"_Branches"+Suffix))#km/s



                                    er_r=1








                                    rads=np.concatenate((radsBase[0],radsBase[1]))
                                    
                                    sigs = np.concatenate((sigsBase[0],sigsBase[1]))
                                    sigs_err = np.concatenate((sigs_err_Base[0],sigs_err_Base[1]))
                                    print(np.shape(rads))

                                    from scipy.optimize import curve_fit

                                    def func(R,a,b):
                                        return a*R**(b)

                                    x   =  rads
                                    y     = sigs
                                    y_err = sigs_err

                                    try:
                                        #(Vars, CoVar) = (curve_fit(func, x, y, sigma=y_err))
                                        (Vars, CoVar) = (curve_fit(func, x, y, ))
                                    except:
                                        (Vars, CoVar) = [np.nan,np.nan],[[np.nan,np.nan],[np.nan,np.nan]]
                                    np.savetxt(os.path.join(savePath,Name+"_CMZ_Coeffs"+Suffix+NM_mod), Vars,fmt='%s')
                                    np.savetxt(os.path.join(savePath,Name+"_CMZ_Coeff_CoVar"+Suffix+NM_mod),CoVar ,fmt='%s')

                                    np.savetxt(os.path.join(savePath,Name+"_CMZ_Fit_weights"+Suffix+NM_mod),[(len(sigs))] ,fmt='%s')
                                    #np.savetxt(CMZName+"_CMZ_Coeffs"+Suffix, Vars2,fmt='%s')
                                    #np.savetxt(CMZName+"_CMZ_Coeff_CoVar"+Suffix,CoVar2 ,fmt='%s')

                                    if Plot:
                                        print(Vars,CoVar)
                                        #gggg=ax.plot(x,func(x,Vars[0],Vars[1]),color="Red",label=CMZLs[i])
                                        #ax.fill_between(x,func(x,Vars[0]+np.sqrt(CoVar[0][0])*er_r,Vars[1]+np.sqrt(CoVar[1][1])*er_r),func(x,Vars[0]-np.sqrt(CoVar[0][0])*er_r,Vars[1]-np.sqrt(CoVar[1][1])*er_r),alpha=.2,color="Red")
                                        #ax.errorbar(x, y, yerr=y_err,lolims=True, uplims=True,fmt="o",color="Blue")
                                        
                                        ax.fill_betweenx(beam_req_line[1][0],beam_req_line[0][0] ,zeroes[0][0], alpha=.1,color="Black")
                                        ax.plot(Beams_s[0][0],Beams_s[1][0], alpha=.1,color="Black")

                                        spec=np.array([[np.linspace(-100,1000,100)],[np.full(100,vel_prime)]])

                                        ax.plot(spec[0][0],spec[1][0], alpha=.1,color="Black")

                                        #ax.scatter(x,y,color="Red",alpha=.51,s=50)



                                        ax.set_xlabel(r'$R_{astrodendro}$ (pc)',fontsize=15)
                                        ax.set_ylabel(r'$\sigma   (km/s)$',fontsize=15)
                                        ax.set_yscale('log')
                                        ax.set_xscale('log')
                                        #string = ("NGC253 (Binned"+Suffix+"): a="+str(round(Vars[0],3))+" +-" + str(round(np.sqrt(CoVar[0][0]),3)) +", b="+str(round(Vars[1],3))+"+-" + str(round(np.sqrt(CoVar[1][1]),3)) )




                                        #axA.annotate(text=string,fontsize=18,xy=(0.01,.01+0.025*Index),xycoords="axes fraction")
                                        ax.set_xlim(0.5,40.0)
                                        ax.set_ylim(0.5,50.0)

                                        #axA.legend(loc='upper left',fontsize=18)

                                        import datetime
                                        now = str(datetime.datetime.now())
                                        string=now+ " "+ "Nico's method, common Gauss Smoothing (No Trunks,  min size=.9pc, delt = 1*m, m=.115K, SNR=5, max_size=21pc, G_width = sqrt(2.5^2-1),NM=true )"
                                        ax.annotate(text=string,fontsize=18,xy=(0.01,1.05),xycoords="axes fraction")
                                        
                                        
                                        ax.set_title(("beam={0},vel_res={1},beam_req={2},pix_thresh={3},abs={4},delta={5},SNR={6}".format(Prime_Beam,vel_prime,beam_req,pix_thresh_factor,Abs_Level,delt_factor,noise_factor*5)), fontsize=14)
                                        if i==5:

                                            def NicoCOCoef(sig10,exp):
                                                return sig10/10**exp
                                            def NicoAErr(Aer,b,bErr):
                                                return np.sqrt(Aer**2/10**(2*b)+b**2*Aer**2*bErr**2/b**(2*b+2))

                                            Nxs=np.linspace(np.nanmin(.5),np.nanmax(20),50)
                                            xs=np.linspace(np.nanmin(.9),np.nanmax(20),50)
                                            NicoCO32 = .72
                                            NicoCOErr32=.03
                                            NicoCO_32= 8.9
                                            NicoCO_32Err= .2
                                            NNicoCO32 = .62
                                            NNicoCOErr32=.01
                                            NNicoCO_32=17.1
                                            NNicoCO_32Err= .1


                                            NNI32 = func(Nxs,NicoCOCoef(NNicoCO_32,NNicoCO32),NNicoCO32)
                                            NNIU32 = func(Nxs,NicoCOCoef(NNicoCO_32+NicoAErr(NNicoCO_32Err,NNicoCO32,NNicoCOErr32)*er_r,NNicoCO32+NNicoCOErr32*er_r),NNicoCO32+ NNicoCOErr32*er_r)
                                            NNIL32 = func(Nxs,NicoCOCoef(NNicoCO_32-NicoAErr(NNicoCO_32Err,NNicoCO32,NNicoCOErr32)*er_r,NNicoCO32-NNicoCOErr32*er_r),NNicoCO32- NNicoCOErr32*er_r)

                                            NCI32 = func(xs,NicoCOCoef(NicoCO_32,NicoCO32),NicoCO32)
                                            NCIU32 = func(xs,NicoCOCoef(NicoCO_32+NicoAErr(NicoCO_32Err,NicoCO32,NicoCOErr32)*er_r,NicoCO32+NicoCOErr32*er_r),NicoCO32+ NicoCOErr32*er_r)
                                            NCIL32 = func(xs,NicoCOCoef(NicoCO_32-NicoAErr(NicoCO_32Err,NicoCO32,NicoCOErr32)*er_r,NicoCO32-NicoCOErr32*er_r),NicoCO32- NicoCOErr32*er_r)



                                            NicoNM32 = ax.plot(xs,NCI32,c='black',label='Krieger CO_3-2 CMZ')
                                            NicoFill32 = ax.fill_between(xs,NCIL32,NCIU32,color="black",alpha=.2)
                                            
                                            
                                            
                                            #cmz bins
                                            xs=np.linspace(np.nanmin(rads),np.nanmax(rads),50)
                                            Bin_s=[]
                                            Bin_s16=[]
                                            Bin_s84=[]
                                            s_err=[]
                                            Bin_rad=[]
                                            bins=[]
                                            '''
                                            bins = np.logspace(np.log10(np.nanmin(rads)),np.log10(np.nanmax(rads)),15)
                                            bins=list(bins)
                                            for lmi in range(len(bins)-1):
                                                SBin=[]
                                                for lmj in range(len(rads)):
                                                    if(rads[lmj]>bins[lmi] and rads[lmj]<bins[lmi+1]):
                                                        SBin.append(sigs[lmj])
                                                if(str(np.nanmean(SBin))!=str(np.nan) and str(np.nanmean(SBin))!= str(np.inf)):
                                                    
                                                    if False:
                                                        Bin_s.append(np.nanmean(SBin))
                                                        s_err.append(np.nanstd(SBin))
                                                    else:
                                                        y_perc = np.percentile(SBin, [16,50,84])
                                                        #Bin_s.append(y_perc[1])
                                                        Bin_s.append(np.nanmean(SBin))
                                                        Bin_s16.append(y_perc[0])
                                                        Bin_s84.append(y_perc[2])
                                                        err= (y_perc[1]-y_perc[0]+ y_perc[2]-y_perc[1])/2#apparently the error
                                                        s_err.append(err)
                                                        #print(y_perc[1],np.nanmean(SBin),"err",err,np.nanstd(SBin))
                                                        #print("#  16th 50th 84th")
                                                        #print(lmi,y_perc[0],y_perc[1],y_perc[2])

                                                else:
                                                    Bin_s.append(np.nan)
                                                    #Ns_err.append(np.nan)
                                                    s_err.append(np.nan)

                                            Bin_s.append(np.nan)
                                            s_err.append(np.nan)#get right shapes
                                            del Bin_s[len(Bin_s)-1]
                                            del s_err[len(s_err)-1]
                                            del bins[len(bins)-1]
                                            bw = (bins[0+1]-bins[0])
                                            '''
                                            def filter_window(x,y,window_min,window_max):
                                                """select values in the window"""
                                                x_window = x[(window_min<x) & (x<window_max)]
                                                y_window = y[(window_min<x) & (x<window_max)]
                                                return x_window,y_window
                                            
                                            
                                            def crossmatch(*args):
                                                """Crossmatch lists for non-finite values.
                                                Parameters
                                                ----------
                                                x : list
                                                y : list
                                                ...
                                                Returns
                                                -------
                                                list, list
                                                    Lists in input order with the non-finite (infinite and NaN) values removed from the list and
                                                    also the corresponding element of the other list.
                                                """
                                                import numpy as np

                                                lists = []
                                                for list in args:
                                                    lists.append( np.array(list) )

                                                selection = np.isfinite(lists[0])
                                                for list in lists[1:]:
                                                    selection = selection & np.isfinite(list)

                                                matched_lists = []
                                                for list in lists:
                                                    matched_lists.append( list[selection] )

                                                return matched_lists
                                            def get_binned_percentiles(x_measure, y_measure, x_step, xdata, ydata):

                                                from astropy.table import Table

                                                def filter_window(x,y,window_min,window_max):
                                                    """select values in the window"""
                                                    x_window = x[(window_min<x) & (x<window_max)]
                                                    y_window = y[(window_min<x) & (x<window_max)]
                                                    return x_window,y_window


                                                # binned percentiles table
                                                bin_perc = Table(names = ['window min','window center','window max','x 16th','x median','x 84th','y 16th','y median','y 84th'],
                                                                 meta = {'x': x_measure, 'y': y_measure, 'x_step': x_step}
                                                                )

                                                x_matched, y_matched = crossmatch(xdata,ydata)

                                                x_min = np.nanmin(x_matched)
                                                x_max = np.nanmax(x_matched)
                                                y_min = np.nanmin(y_matched)
                                                y_max = np.nanmax(y_matched)

                                                for log_window_cen in np.arange(np.log10(x_min), np.log10(x_max)+x_step, x_step):
                                                    log_window_min = log_window_cen-x_step/2.
                                                    log_window_max = log_window_cen+x_step/2.
                                                    window_min = np.power(10,log_window_min)
                                                    window_cen = np.power(10,log_window_cen)
                                                    window_max = np.power(10,log_window_max)
                                                    x_window,y_window = filter_window(x_matched,y_matched,window_min,window_max)

                                                    # percentiles only make sense when at least two measurements are available
                                                    if len(x_window)>1:
                                                        x_perc = np.percentile(x_window, [16,50,84])
                                                        y_perc = np.percentile(y_window, [16,50,84])
                                                        bin_perc.add_row( flatten([window_min,window_cen,window_max,x_perc,y_perc]) )

                                                return bin_perc
                                            def flatten(x):
                                                """Flatten list of lists.
                                                Parameters
                                                ----------
                                                x : list,tuple
                                                    List of lists of lists of ...
                                                    Can contain a mixture of lists and values.
                                                Returns
                                                -------
                                                list
                                                    Flattened list.
                                                """
                                                import collections
                                                if isinstance(x, collections.Iterable):
                                                    return [a for i in x for a in flatten(i)]
                                                else:
                                                    return [x]



                                                # binned percentiles table
                                                bin_perc = Table(names = ['window min','window center','window max','x 16th','x median','x 84th','y 16th','y median','y 84th'],
                                                                 meta = {'x': x_measure, 'y': y_measure, 'x_step': x_step}
                                                                )

                                                x_matched, y_matched = crossmatch(xdata,ydata)

                                                x_min = np.nanmin(x_matched)
                                                x_max = np.nanmax(x_matched)
                                                y_min = np.nanmin(y_matched)
                                                y_max = np.nanmax(y_matched)

                                                for log_window_cen in np.arange(np.log10(x_min), np.log10(x_max)+x_step, x_step):
                                                    log_window_min = log_window_cen-x_step/2.
                                                    log_window_max = log_window_cen+x_step/2.
                                                    window_min = np.power(10,log_window_min)
                                                    window_cen = np.power(10,log_window_cen)
                                                    window_max = np.power(10,log_window_max)
                                                    x_window,y_window = filter_window(x_matched,y_matched,window_min,window_max)

                                                    # percentiles only make sense when at least two measurements are available
                                                    if len(x_window)>1:
                                                        x_perc = np.percentile(x_window, [16,50,84])
                                                        y_perc = np.percentile(y_window, [16,50,84])
                                                        bin_perc.add_row( flatten([window_min,window_cen,window_max,x_perc,y_perc]) )

                                                return bin_perc
                                            x_matched, y_matched = crossmatch(rads,sigs)

                                            x_step=.1
                                            x_min = np.nanmin(rads)
                                            x_max = np.nanmax(rads)
                                            #y_min = np.nanmin(y_matched)
                                            #y_max = np.nanmax(y_matched)

                                            for log_window_cen in np.arange(np.log10(x_min), np.log10(x_max)+x_step, x_step):
                                                log_window_min = log_window_cen-x_step/2.
                                                log_window_max = log_window_cen+x_step/2.
                                                window_min = np.power(10,log_window_min)
                                                window_cen = np.power(10,log_window_cen)
                                                window_max = np.power(10,log_window_max)
                                                x_window,y_window = filter_window(x_matched,y_matched,window_min,window_max)

                                                # percentiles only make sense when at least two measurements are available
                                                if len(x_window)>1:
                                                    x_perc = np.percentile(x_window, [16,50,84])
                                                    y_perc = np.percentile(y_window, [16,50,84])
                                                    #bin_perc.add_row( flatten([window_min,window_cen,window_max,x_perc,y_perc]) )
                                                    bins.append(window_cen)
                                                    Bin_s.append(y_perc[1])
                                                    #Bin_s.append(np.nanmean(SBin))
                                                    Bin_s16.append(y_perc[0])
                                                    Bin_s84.append(y_perc[2])
                                                    err= (y_perc[1]-y_perc[0]+ y_perc[2]-y_perc[1])/2#apparently the error
                                                    s_err.append(err)
                                                    #print(y_perc[1],np.nanmean(SBin),"err",err,np.nanstd(SBin))
                                                    #print("#  16th 50th 84th")
                                                    #print(lmi,y_perc[0],y_perc[1],y_perc[2])
                                            er_r=1
                                            
                                            '''
                                            xp1BinC = ax.plot([bins[0],bins[1]], [Bin_s[0],Bin_s[0]],c="blue",alpha=1,label=("Binned CMZ CO 3-2 fit"))
                                            '''
                                            #xp1BinC = ax.plot([bins[0],bins[1]], [Bin_s[0],Bin_s[0]],c="blue",alpha=1,label=("Binned CMZ CO 3-2 fit"))
                                            '''
                                            for lmi in range(len(bins)):
                                                if((s_err[lmi])==0.0):            
                                                    s_err[lmi] = Bin_s[lmi]#Only one structure in a bin gives a large error
                                            '''
                                            '''


                                            for w in range(len(bins)-1):
                                                bw = (bins[w+1]-bins[w])
                                                xp1BinC = ax.plot([bins[w],bins[w+1]], [Bin_s[w],Bin_s[w]],c="blue",alpha=1)
                                                
                                                ax.fill_between([bins[w],bins[w+1]], Bin_s16[w],Bin_s84[w], color='blue', alpha=.3)
                                                
                                                #ax.add_patch(Rectangle(
                                                #xy=(bins[w], Bin_s[w]-s_err[w]*er_r) ,width=bw, height=s_err[w]*er_r*2,
                                                #linewidth=1, color='blue', fill=True,alpha=.3,))
                                                
                                            Bin_Centers=[]
                                            for lmi in range(len(bins)-1):
                                                Bin_Centers.append((bins[lmi]+bins[lmi+1])/2)
        
                                            del Bin_s[len(Bin_s)-1]
                                            del s_err[len(s_err)-1]

                                            for lmi in range(len(bins)-1):
                                                if(lmi>len(Bin_s)-1):
                                                    break
                                                #print(lmi,len(NBin_s),len(Bin_Centers))
                                                if(str(Bin_s[lmi])==str(np.nan)):
                                                    del Bin_s[lmi]
                                                    del Bin_Centers[lmi]
                                                    del s_err[lmi]
                                                elif((s_err[lmi])==0.0):
                                                    s_err[lmi] = Bin_s[lmi]#Only one structure in a bin gives a large error
                                            #(popt_bin, pcov_bin) = (curve_fit(func, Bin_Centers, Bin_s,sigma=s_err))#Should 'perfectly' replcate krieger
                                            '''
                                            '''(popt_bin, pcov_bin) = (curve_fit(func, Bin_Centers, Bin_s,sigma=s_err))#Should 'perfectly' replcate krieger
                                            '''
                                            
                                            x     = np.array(bins)
                                            y     = np.array(Bin_s)
                                            y_p16 = np.array(Bin_s16)
                                            y_p84 = np.array(Bin_s84)
                                            print(y)
                                            print(y_p16)
                                            print(y_p84)
                                            print(np.shape(y))
                                            print(np.shape(y_p16))
                                            print(np.shape(y_p84))
                                            print(type(y))
                                            print(type(y_p16))
                                            print(type(y_p84))
                                            
                                            print(y-y_p16)
                                            print(y_p84-y)
                                            y_err = np.nanmean([y-y_p16, y_p84-y], axis=0)

                                            fit = fit_MCMC(log_x     = np.log10(x),
                                                           log_y     = np.log10(y),
                                                           log_x_err = [0.05 for x in x],            # half bin width
                                                           log_y_err = np.log10(y_err),
                                                           source="GC", line="co 3-2",
                                                           xlabel =" R_measure.replace('_',' ')",
                                                           ylabel = "lw_measure.replace('%',r'\%')",
                                                           savepath = ""
                                                          )
                                            
                                            (popt_bin, pcov_bin) = [np.nan,np.nan],[[np.nan,np.nan],[np.nan,np.nan]]
                                            
                                            
                                            bins = get_binned_percentiles("R_measure", "lw_measure",
                                              x_step = 0.1,
                                              xdata  = x_matched,
                                              ydata  = y_matched
                                             )
                                            
                                            print("j",bins)
                                            i = 0
                                            for bin in bins:
                                                window = [bin['window min'], bin['window max']]
                                                window_cen = bin['window center']
                                                median = [bin['y median'], bin['y median']]
                                                perc16 = [bin['y 16th'], bin['y 16th']]
                                                perc84 = [bin['y 84th'], bin['y 84th']]

                                                ax.plot(window, median,                 color="purple",                                zorder=3, label='')
                                                ax.fill_between(window, perc16, perc84, facecolor="purple", edgecolor=None, alpha=0.5, zorder=2, label='')
                                                i += 1

                                            x = np.logspace(np.log10(np.min(bins['window min'])), np.log10(np.max(bins['window max'])), 100)
                                            if gal =="GC":
                                                ax.plot(x, powlaw(x,fit.a['ls'][0],np.power(10,fit.c['ls'][0])), color="blue", zorder=4,label="co 3-2 cmz with kr analysis")
                                            else:
                                                ax.plot(x, powlaw(x,fit.a['ls'][0],np.power(10,fit.c['ls'][0])), color="blue", zorder=4,label="co 3-2 NGC253 with kr analysis")
                                            
                                            
                                            print("AAAAA", fit.a['ls'][0]     ,    fit.a['ls'][1]   ,      fit.y10['ls'][0], fit.y10['ls'][1])
                                            
                                            
                                            #gggghg=ax.plot(Bin_Centers,func(Bin_Centers,popt_bin[0],popt_bin[1]),color="blue",label="Binned fit ")
                                            #fbbff=ax.fill_between(Bin_Centers,func(Bin_Centers,popt_bin[0]+np.sqrt(pcov_bin[0][0])*er_r,popt_bin[1]+np.sqrt(pcov_bin[1][1])*er_r),func(Bin_Centers,popt_bin[0]-np.sqrt(pcov_bin[0][0])*er_r,popt_bin[1]-np.sqrt(pcov_bin[1][1])*er_r),alpha=.2,color="blue")



                                    ax.legend(loc='upper left',fontsize=18)

                                    print(i,kl,km,k3,k4,k5,k6,k7,"Clear")
                                except Exception as e:
                                    print(e)
                                    print(i,kl,km,k3,k4,k5,k6,k7,"F")
                                    print(Name)
                                    print("Failed")
                                    print("-"*60)
                                    traceback.print_exc(file=sys.stdout)
                                if Plot:
                                
                                    plt.tight_layout(pad=5, w_pad=5 ,h_pad=50.0)

                                    pylab.show()
                                    fname = "All_"+str(Prime_Beam)+"_beam_"+str(vel_prime) +"_vel_res_"+Suffix+"_.jpeg"
                                    fig.savefig(bbox_inches='tight',fname=fname)
                                    
                                    
                                    